In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
import jsonlines
import glob
from tqdm.auto import tqdm
from transformers import AutoTokenizer

In [3]:
fnames = glob.glob('../data_large/pantip-large/*.jsonl')
len(fnames)

72

In [3]:
# %%time

# content_dfs = []
# comment_dfs = []
# for fname in tqdm(fnames):
#     with jsonlines.open(fname) as reader:
#         for obj in tqdm(reader):
#             if 'comment' in obj.keys():
#                 d = {
#                     'datasource_url': obj['datasource_url'],
#                     'content_id': obj['content_id'],
#                     'comment_id': obj['comment_id'],
#                     'order_comment': obj['order_comment'],
#                     'type':obj['type'],
#                     'comment': obj['comment'],
#                     'like_score': obj['like_score'],
#                     'feel_heart': obj['feel_heart'],
#                     'feel_laugh': obj['feel_laugh'],
#                     'feel_love': obj['feel_love'],
#                     'feel_sad': obj['feel_sad'],
#                     'feel_horror': obj['feel_horror'],
#                     'feel_wow': obj['feel_wow'],
#                 }
#                 comment_dfs.append(d)
#             elif ('content' in obj.keys()) and (obj['total_field'] > 1):
#                 d = {
#                     'datasource_url': obj['datasource_url'],
#                     'content_id': obj['content_id'],
#                     'type':obj['type'],
#                     'title': obj['title'],
#                     'body': obj['content'],
#                     'total_field': obj['total_field'],
#                     'tags': obj['tags'],
#                     'like_score': obj['like_score'],
#                     'feel_heart': obj['feel_heart'],
#                     'feel_laugh': obj['feel_laugh'],
#                     'feel_love': obj['feel_love'],
#                     'feel_sad': obj['feel_sad'],
#                     'feel_horror': obj['feel_horror'],
#                     'feel_wow': obj['feel_wow'],
#                 }
#                 content_dfs.append(d)    

# comment_df = pd.DataFrame(comment_dfs)
# content_df = pd.DataFrame(content_dfs)

In [4]:
# content_df = content_df.drop_duplicates()
# #remove threads with different snapshots
# content_df['rnk'] = content_df.sort_values(['total_field',],ascending=[False]) \
#              .groupby(['content_id']) \
#              .cumcount() + 1
# content_df = content_df[content_df.rnk==1]
# content_df.to_csv('../data_large/content_df.csv', index=None)


# comment_df = comment_df.drop_duplicates()
# comment_df['rnk'] = comment_df.groupby(['content_id','comment_id']).cumcount() + 1
# comment_df = comment_df[comment_df.rnk==1]
# comment_df.to_csv('../data_large/comment_df.csv', index=None)


# comment_df = pd.read_csv('../data_large/comment_df.csv')
# content_df = pd.read_csv('../data_large/content_df.csv')

In [5]:
# #check if content deduplicated successfully
# content_df.shape, content_df.content_id.nunique(), content_df.drop_duplicates().shape

In [6]:
# #check if content deduplicated successfully
# comment_df.shape, comment_df.comment_id.nunique(), comment_df.drop_duplicates().shape

In [253]:
# all_df = content_df.merge(comment_df, on='content_id')
# all_df = all_df.drop(['rnk_x','rnk_y','datasource_url_y'],1)
#filter out later sub-comments
# all_df = all_df[all_df.order_comment.map(lambda x: '-' not in str(x))]
#create interact count
# all_df['nb_interact'] = all_df[['like_score_y',
#        'feel_heart_y', 'feel_laugh_y', 'feel_love_y', 'feel_sad_y',
#        'feel_horror_y', 'feel_wow_y']].sum(1)

#only keep top 5 comments with highest interacts
# all_df['rnk'] = all_df.sort_values(['nb_interact','like_score_y','feel_heart_y'], \
#              ascending=False) \
#              .groupby(['content_id']) \
#              .cumcount() + 1
# all_df = all_df[all_df.rnk<=5].drop('rnk',1)

# #filter less than 2 comment
# all_df = all_df[all_df.total_field>1]

# #pick only question in title
# question_signals = ['ใคร',
#                     'ทำไม',
#                     'อะไร',
#                     'ไหน',
#                     'ไหม','มั้ย',
#                     'เท่าไร','ไหร่','กี่',
#                     'อย่างไร','ยังไง',
#                     'หรือ','หรอ','เรอะ']
# all_df = all_df[all_df.title.map(lambda x: sum(i in x for i in question_signals)>0)]

# #remove edit artifacts
# all_df['title'] = all_df.title.map(lambda x: x.split('แก้ไขข้อความเมื่อ')[0])
# all_df['body'] = all_df.body.map(lambda x: x.split('แก้ไขข้อความเมื่อ')[0])
# all_df['comment'] = all_df.comment.map(lambda x: x.split('แก้ไขข้อความเมื่อ')[0])

# #replace URL
# import re
# def replace_url(text): return re.sub(r'http\S+|www\S+', '[URL]', text)

# all_df['title'] = all_df.title.map(replace_url)
# all_df['body'] = all_df.body.map(replace_url)
# all_df['comment'] = all_df.comment.map(replace_url)

# #remove spoiler tags
# all_df['body'] = all_df.body.map(lambda x: str(x).replace('[Spoil] คลิกเพื่อดูข้อความที่ซ่อนไว้',''))
# all_df['comment'] = all_df.comment.map(lambda x: str(x).replace('[Spoil] คลิกเพื่อดูข้อความที่ซ่อนไว้',''))

# #drop na
# all_df = all_df.dropna()

#count tokens
# tokenizer_mgpt = AutoTokenizer.from_pretrained('sberbank-ai/mGPT')

# #title
# title_tokens = tokenizer_mgpt(all_df.title.tolist())
# all_df['title_tokens'] = [len(i) for i in title_tokens.input_ids]

# #body
# batch_size = 100_000
# toks = []
# for i in tqdm(range(0, len(all_df), batch_size)):
#     body_tokens = tokenizer_mgpt(all_df.body.tolist()[i:i+batch_size])
#     toks+= [len(i) for i in body_tokens.input_ids]
# all_df['body_tokens'] = toks

# #comment
# batch_size = 100_000
# toks = []
# for i in tqdm(range(0, len(all_df), batch_size)):
#     comment_tokens = tokenizer_mgpt(all_df.comment.tolist()[i:i+batch_size])
#     toks+= [len(i) for i in comment_tokens.input_ids]
# all_df['comment_tokens'] = toks

# #filter comments
# #min tok 12 (10th percentile) to filter out ขอบคุณ
# #max tok 512 (97th percentile) to filter out too detailed answers
# all_df = all_df[(all_df.comment_tokens>=12)&(all_df.comment_tokens<=512)]

# #filter title
# #min tok 16 (5th percentile) to filter out super short titles e.g. ทำไม?
# #max tok 75 (99th percentile) to filter out too long titles with too many special characters/emojis
# all_df = all_df[(all_df.title_tokens>=12)&(all_df.title_tokens<=75)]

# #filter prompt
# #min tok 48 (5th percentile) to filter out questions that are too random
# #max tok 512 (90th percentile) just to fit 1024 prompt+demo
# all_df = all_df[(all_df.prompt_tokens>=48)&(all_df.prompt_tokens<=512)]

# #filter out thread with less than 2 comments
# more_than_one = all_df.content_id.value_counts().reset_index()
# more_than_one = more_than_one[more_than_one.content_id>1]
# more_than_one.columns = ['content_id','nb_comments']
# all_df = all_df.merge(more_than_one,on='content_id',how='inner')

# #sort by nb_interact then comment_tokens; topmost has label 1, otherwise 0
# all_df['rnk'] = all_df.sort_values(['nb_interact','comment_tokens'], ascending=[False,False]) \
#              .groupby(['content_id']) \
#              .cumcount() + 1
# all_df['preference_label'] = all_df['rnk'].map(lambda x: 1 if x==1 else 0)

# #add strict labels for only threads that top comments indeed have more interacts (not just longer, no ties)
# rank_1 = all_df[all_df.rnk==1][['content_id','title','body','comment','nb_interact']]
# rank_2 = all_df[all_df.rnk==2][['content_id','comment','nb_interact']]
# rank_combined = rank_1.merge(rank_2, on='content_id')
# rank_combined['strict'] = 1
# rank_combined = rank_combined[rank_combined.nb_interact_x>rank_combined.nb_interact_y]
# rank_combined = rank_combined[['content_id','strict']]
# all_df = all_df.merge(rank_combined, on='content_id',how='left')
# all_df['strict'] = all_df.strict.fillna(0)

all_df.to_csv('../data_large/all_df.csv', index=None)
all_df.shape, all_df.content_id.nunique()

((3510036, 34), 935910)

In [254]:
all_df = pd.read_csv('../data_large/all_df.csv')

In [255]:
all_df.shape, all_df.content_id.nunique()

((3510036, 34), 935910)

## Pick Sequence Length for Comments

In [130]:
all_df.comment_tokens.describe().round(3)

count    4304496.000
mean     91.042     
std      90.472     
min      0.000      
25%      30.000     
50%      58.000     
75%      116.000    
max      512.000    
Name: comment_tokens, dtype: float64

In [129]:
all_df.comment_tokens.quantile([i/100 for i in range(1,100)])

0.01    12.0 
0.02    13.0 
0.03    14.0 
0.04    14.0 
0.05    15.0 
        ...  
0.95    292.0
0.96    317.0
0.97    347.0
0.98    386.0
0.99    437.0
Name: comment_tokens, Length: 99, dtype: float64

In [80]:
all_df[all_df.comment_tokens<=1024][['content_id','title','comment','comment_tokens']]\
    .sort_values('comment_tokens',ascending=False).head(30)

,content_id,title,comment,comment_tokens
3420406,32627079,มีใครเป็นเหมือนกันไหม ไม่กล้าซื้อคอนโด,"ส่วนตัว อายุ กำลังจะ 30 ตัวคนเดียว- อายุผมพึ่ง 30 มาไม่กี่วัน ตัวคนเดียว (เพิ่งเลิกกับแฟนเมื่อไม่กี่วันเช่นกัน)อยู่ กทม มา ตั้งแต่อายุ 14 เช่าอยู่ตลอด- อยู่ กทม. มาประมาณ 6-7 ปี ตั้งแต่เรียนจบมหาวิทยาลัยก็มาอยู่ กทม.ทุกวันนี้ เงินเดือน ประมาณ ประมาณ 50k-60k แต่ยังเช่าอยู่เดือนละ 5000 ไม่กล้าซื้อคอนโด เช่าอยู่มา 4-5 ปีแล้ว- ผมก็เช่าอยู่มา 6-7 ปีนั่นแหละ เดือนละ เกือบ 6,000 รวมค่าหอ ค่าน้ำ ไฟแล้ว รายได้ก็พอๆกับคุณเลย 5 กว่าๆเพราะกลัวดอกเบี้ยบ้าน- อันนี้ไม่ค่อยกลัว กลัวเสียค่าหอฟรีมากกว่า รู้สึกแบบนั้นจริงๆนะ ทุกวันนี้ดูในใบเสร็จ ดอกเบี้ยเดือนละประมาณ 5,000 อ่ะค่าคอนโด แต่ก็รู้สึกดีกว่าเช่าอยู่มากกลัวย้ายที่ทำงาน เดินทางไม่สะดวก (แต่ตั้งใจว่า จะอยู่ กทม ตลอดไป)- เรื่องนี้ไม่ค่อยกังวล เพราะทำงานมา 4 ที่ เน้นติดรถไฟฟ้า ถ้าไม่ได้งานแถวรถไฟฟ้าก็ไม่เลือกไม่กล้าซื้อกลัวเป็นภาระถ้าอยู่ไกลๆ- อันนี้ไม่เคยคิดเหมือนกันครับ รู้สึกแค่ว่าอยู่มาตั้งหลายปี เลยไม่คิดว่าจะไปอยู่ไหนไกลๆได้อีกถามคนที่ลังเล หรือเคยลังเล แบบนี้ ทุกวันนี้ซื้อหรือยัง เพราะอะไรถึงซื้อ- ทุกวันนี้ซื้อแล้วครับ ซื้อมากำลังจะครบ 1 ปี เพราะซื้อและเข้าอยู่ เดือน มกรา 57- ซื้อเพราะว่า รู้สึกว่าอยู่หอมันเสียเปล่ามากๆ มองย้อนหลังไป เสียค่าห้องเดือนละ 5000 ถ้า 1 ปีก็ 60,000 บาท ทั้งหมด 6 ปีก็ 360,000 บาท โดยที่เทียบกับตอนนี้ถึงจะเสียดอกเบี้ยแพง แต่ถ้าขายไป ถึงขาดทุนก็ยังได้เงินต้นคืนบ้าง (แต่เท่าที่เห็น ส่วนใหญ่ถ้าไม่ได้ร้อนเงิน ขายไปก็กำไรกันแทบจะทุกคนนะ)ความรู้สึกส่วนตัว มันทำให้เรารู้สึกว่านี่คือบ้าน อยากจะทำอะไร ขีดเขียนตกแต่งก็ทำได้ เพราะเป็นทรัพย์สินของเราจริงๆความกังวล เรื่องเพื่อนบ้าน นิติ สังคมในคอนโด เป็นเรื่องที่กังวลที่สุด แต่ตอนนี้อยู่มาจะปี ถือว่าโชคดี เพราะได้ห้องติดๆกันเป็นคนที่ค่อนข้างมารยาทดี จึงไม่เคยมีปัญหาใดๆเลย (หรือเพราะผมไม่ใช่คนเรื่องมากก็ไม่รู้นะ)พิมพ์เพลินเลย พอเป็นเรื่องใกล้ตัว 555+",1024
2108229,35386017,อะไรคือสาเหตุที่ทำให้ห้องนี้มีแต่เกาหลี,แล้วอะไรคือสาเหตุที่ทำให้คนที่เล่นพันทิปส่วนใหญ่ ไม่รู้จักวิธีปิดแทค ?อันนี้สิเราอยากรู้ บางคนที่มาบ่น มากระแนะกระแหนมักจะเคลมตัวว่าเล่นมานาน (แต่เล่นอยู่ห้องอื่นนะ)จะเข้ามาหากระทู้ดีๆอ่านก็หาไม่เจอ มีแต่กระทู้เกาหลีเต็มไป นี่เฉลิมกรุงโซลรึเปล่า .. คือเราแปลกใจเลยอ่ะก็ในเมื่อเล่นมานานแล้วทำไมถึงไม่รู้ว่าพันทิปปิดเแทคได้ ฟังชั่นมันก็ขึ้นอยู่บนจอนะ ทำไมไม่เรียนรู้กันบ้างพอบอกไปก็ตอบมาแบบ ไม่รู้ ทำไมเป็น ใช้ไม่เป็น คือไม่รู้ก็บอก จะได้สอน ไม่ได้มาด่ากันก่อนแบบนี้เหมือนกระทู้ดราม่าแล้วคนชอบเม้นว่า'ปัก'อ่ะ ปักกันจนเข้าไปแล้วไม่ได้อ่านข้อมูลอะไรเลย เพราะปักกันเป็นร้อยๆทั้งที่ฟังค์ชั่นติดธงมันก็ขึ้นให้เห็นอยู่ ทำไมถึงไม่เรียนรู้ที่จะใช้กัน คือโทษทุกอย่างยกเว้นตัวเองยิ่งห้องเฉลิมกรุง อะไรในห้องที่แย่ๆ โยนมาให้แฟนคลับเกาหลีได้เลยค่ะ ผิดทุกอย่าง แก้ตัวไม่ได้ด้วย เบื่ออ่ะยิ่งตอนexoมาไทยครั้งก่อนนี่ชัดมาก สงสารอซอ.ดีๆในพันทิปเลย โดนสวมรอยเสี้ยมเป็นสิบๆกระทู้แนะนำเลยนะคะ ปิดแทคไปค่ะ มันช่วยได้แบบ90%เลย แต่สิ่งที่ต้องยอมรับเมื่อปิดไปแล้วคือคุณแทบจะหากระทู้ดีๆไม่เจอ เพราะไม่มีคนตั้ง มีแต่กระทู้ถามเพลง กระทู้ถามเรื่องนู้นเรื่องนี้ และไม่มีคนตอบด้วยนานๆกระทู้ดีจะมาทีนึง เมื่อเช้าก็มีคนมาแนวนี้แหล่ะ ทำไมไม่มีกระทู้เก่ง ธชย เอ๊าา .. ก็แล้วทำไมไม่ตั้งอ่ะมาบ่นว่าไม่มีใครตั้ง แต่ตัวเองรู้ข่าวแล้วไม่ยอมมาตั้ง แล้วมาโทษแฟนคลับเกาหลี แฟร์ตรงไหนอ่ะแล้วสรุป คนที่ตั้งกระทู้ให้คือคุณGirl on Top 1ในแฟนคลับเกาหลีของเฉลิมกรุงนะคะเลิกตำหนิกันเถอะ คนที่เต็มไปด้วยอคติทั้งหลาย อยากอ่านกระทู้ดี ก็ขยันมาผลัดกันตั้ง ถ้ามีคือโหวตแน่นอนอยู่แล้วอ่ะแฟนฝั่งตะวันตกเค้าก็มาตั้งกระทู้MVใหม่ๆ เพลงใหม่กันตลอด เช่น กระทู้เพลงHelloของAdeleขึ้นแนะนำคนห้องอื่นมาด่าแฟนคลับเกาหลีแล้วบ่นว่าทำไมหากระทู้เพลงAdeleไม่เจอเลย ตลกมากค่ะ อคติสุด ;'(,1024
1950340,36791978,เทป ที่นี่หมอชิต ของผลิตโชค ทำให้เข้าใจอะไรมากขึ้น,เอาตรงๆนะคะ ทั้งในพันทิป และทวิตเตอร์ มันก็มีคนที่เห็นด้วย และไม่เห็นด้วย ในการปรับตัว คนที่เห็นด้วยก็คือ คนที่ชอบตัวตนเค้า แบบนี้ดีแล้ว โชว์คือปรับได้ เค้าต้องดีกว่านี้แน่ ส่วนคนที่อยากให้ปรับ ก็คือคนที่ก็ชอบเค้าเหมือนกัน ก็รักเค้าเหมือนเดิม แต่คงอยากให้เค้ามีแฟนคลับเพิ่มมากขึ้นส่วนสำหรับตัวเรา ที่เรามองเห็นเค้า ในทุกวันนี้ คือพัฒนาการที่ดีขึ้นเรื่อยๆ เราว่ามันไม่ดรอปลงเลย การแก้ปัญหาหน้างาน โดยเฉพาะตัวเค้าคือดีมาก ภาพรวมคือดี แม้มันไม่ 100% บ้างแต่มันก็ออกมาดี น

In [81]:
all_df[all_df.comment_tokens<=512][['content_id','title','comment','comment_tokens']]\
    .sort_values('comment_tokens',ascending=False).head(30)

,content_id,title,comment,comment_tokens
3103979,30884204,จะเริ่มเก็บออมไว้ในหุ้นต้องทำยังไงบ่างหลอค่ะ,"ถ้าคิดว่าจะ ""ออม"" ให้เปิดบัญชีเงินฝากกับธนาคารครับ จะออมทรัพย์ หรือฝากประจำก็แล้วแต่ ความเสี่ยงต่ำ ผลตอบแทนแน่นอน เหมาะสำหรับ ""ออม"" ไว้ใช้ยามฉุกเฉินแต่การเอาเงินไปซื้อหุ้น ซื้อกองทุน ไม่เรียกว่าออมครับ เพราะมีความเสี่ยงมากกกกกกกกก และอย่าไปเชื่อสโลแกน ""ให้เงินทำงานแทนคุณ"" เพราะยังไงคุณต้องเจียดเวลามาดูแลพอร์ตของคุณ ต้องแน่ใจด้วยว่า เงินก้อนนี้ถ้าเกิดเจ๊งขึ้นมา คุณยังมีกินอยู่ได้ นั่นคือมีเงินออมมากพอสมควรแล้วต้องทำความเข้าใจความแตกต่างของคำว่า ""ออม"" และ ""ลงทุน"" ก่อน เดี๋ยวนี้สับสนกันไปหมดแล้วสถานะการเงินมีลำดับขั้น1. ไม่มีหนี้ ถ้ามีหนี้อยู่ ให้หาทางหรือรอให้ใช้หนี้ให้หมดก่อนแล้วค่อยคิดมาออม ยังไม่ต้องคิดเรื่องลงทุนซื้อหุ้น อะไรทั้งสิ้น2. มีเงินออมแบบว่า ตกงาน ธุรกิจส่วนตัวเจ๊ง อยู่เฉยๆ และเผื่อแผ่ไปถึงพ่อแม่ หรือถ้ามีลูก ก็รวมลูกด้วย โดยไม่ทำอะไรเลยได้อย่างน้อย 1 ปี (อย่างแย่สุด)3. เหลือจากนั้นค่อยคิดลงทุนหาผลตอบแทน",512
2289629,33491244,Taylor Swift นี่ถือว่าดังมากไหม,"อั้ลบั้มใหม่ขายได้เกิน 6 ล้านแผ่นในยุคที่สามารถหาโหลดตามเว็บได้ คิดว่าคนซื้อต้องชอบจริงๆ ผมก็เป็น 1 ใน 6 ล้านกว่าๆในปี 2014 เกือบจะไม่ได้เป็นอัลบั้มที่ขายดีที่สุดแล้วเพราะเล่นเปล่อยตอนเดือนตุลา แต่ก็ทำยอดแซง OST frozen ได้แบบเฉียดฉิวปล่อยเพลงแล้วขึ้น ที่ 1 ใน Billboard 2 เพลง ในอัลบั้มเดียวกัน โดยการปล่อยแค่ 2 เพลง ได้ที่ 1 ทั้งคู่Red tour รอบโลกในปี 2013-2014 ขายตั๋วหมดทุกที่ ประเทศไทยก็ขายหมดแต่ไม่ได้ดู 5555+MV Blank space มียอดวิว 500 ล้าน เร็วที่สุดในโลก ใช้เวลา 94 เร็วกว่ากัมนังอีก และ เป็น MVหญิงที่มียอดไลท์เยอะที่สุดได้ขึ้นปกนิตยสาร TIME เพราะเป็นกบฏต่อวงการเพลงดิจิตอล 555 เพราะคิดว่ามันไม่ยุติธรรมลองไปหาอ่านดูTaylor Swift ถูกยกให้เป็นคนดังที่ใจบุญมากที่สุดแห่งปี 2012-2014ถามว่าดังกว่า Katy perry , Lady Gaga , Beyonce ถ้านับตั้งแต่ ปี2014-ตอนนี้ ผมกล้าตอบว่าดังกว่า แต่ถ้าก่อนปี 2014 ผมว่าคงไม่ดังกว่าตอบในฐานะ Swiftie อาจจะเวอร์ๆไปหน่อยแต่ ข้อมูลนี้ของจริงทุกอย่างนะเช็คได้ ผมติดตามมาตั้ง 1 ปีละ",512
1489658,38688098,คนอายุ 60 ที่ไม่ได้ทำงานประจำ ไม่ได้ขายของ ไม่ได้ทำงานอะไรเลย ใช้ชีวิตยังไง แล้วคิดยังไงกับวันที่เหลืออยู่คะ,อยู่ในสวนมะหร้าวแต่ไม่ได้ทำสวนมะพร้าวเพราะมีคนสวนเก่าแก่ดูแลให้อยู่แล้วชีวิตปัจจุบันสะดวกกว่าเมื่ออยู่กทม.มาก รถไม่ติด อากาศดี และไม่จำเป็นต้องเหนื่อยขับรถเข้าตัวจังหวัด 20-30 กม.เพื่อไปซื้อของ สั่งเอาจากออนไลน์มีของทุกอย่าง ที่สวนกว้างขวางถึง10ไร่จึงทำโรงรถขนาดใหญ่จอดรถได้สิบกว่าคัน ซื้อรถยุโรปเก่าๆมาซ่อมเป็นงานอดิเรกที่อยากทำมานานแล้วแต่ไม่มีเวลาและสถานที่ มีลิฟต์ยกรถ เครื่องมือช่างคุณภาพกลางๆจากจีนแต่ราคาถูกมาก ซื้อมาเก็บสะสมไว้จนสามารถเปิดอู่ซ่อมรถได้เลย ยิ่งปัจจุบัน3BB มาติดตั้งเน็ตแบบไฟเบอร์ให้ ไม่ต้องใช้เน็ตจากมือถือหรือไปร้านเน็ตอีกต่อไปเจ็บป่วยไม่สบายก็มี ส่งเสริมสุขภาพตำบลอยู่ใกล้บ้าน ห่างไป7-8 กม.ก็มีรพ.ชุมชน ถ้าจำเป็นต้องใช้แพทย์เฉพาะทางถึงต้องไปรพ.ประจำจังหวัดแต่ที่แตกต่างไปจากกทม.จะเป็นเพราะบ้านแต่หลังห่างกันมากจึงเงียบๆหน่อย แต่โชคดีที่ชุมชนแถวนี้ชาวบ้านมีรายได้จากสวนทำให้ไม่ค่อยมีข่าวโจรขโมย,512
620922,36575236,พี่น้องต้องช่วยคนที่มาอยู่กับพ่อแม่(อายุ85-86)ด้วยเหรอ?,"เข้าใจละ อ่านดีๆ จขกท. โพสต์ในเชิง ""ดัก"" ไม่ใช่ จขกท. พลาดนะ แต่ ฉลาดในการนำเสนอมากแถม... สังคมในพันทิปตั้งสติอีกรอบจขกท. คือ คนที่ลาออกจากงานเพื่อมาดูแลพ่อแม่แต่เขียนกระทู้ในมุมกลับกันคำพูดที่ดูไร้มนุษยธรรม เป็นคำพูดที่พี่น้องกล่าว ต่อ จขกท.เช่น 3 ปีให้เงิน 300, ก็เห็นไปเที่ยวลาว, กินข้าวกับพ่อแม่จะเอาอะไรอีกแต่ถ้า จขกท. เขียนในเชิงบ่นจากคนเสียสละ คนมาตอบจะเป็นอีกอารมณ์หนึ่ง เด่นะ ""ทำดีแล้ว จขกท. ใครไม่เห็นเราเห็น"" ""คุณเป็นลูกกตัญญูแล้ว เชื่อสิไม่มีลูกกตัญญูไหน ไม่เจริญหรอก""ซึ่ง... จขกท. นี้ไม่ได้ต้องการความเห็นใจแบบนั้น แต่อยากให้สังคมตัดสินจากการกระทำของพี่น้อง ว่าการที่พี่น้องทำแบบนี้ สังคมพันทิปมีความเห็นไม่พอใจเพียงไหนจขกท. คุณเขียนดักได้ฉลาดมาก แต่... เพราะคุณไม่ได้คิดในมุมพี่น้องจริงๆ จึงดูเขียนงงๆถ้าให้ดี ไป edit ในต้นกระทู้ด้วย เฉลยว่าเป็นข้อความดัก จะได้กันคนอ่านแต่ต้นกระทู้ กับชื่อกระทู้ มาตอบ",512
1091073,33899083,ประจำเดือนไม่มา เกิดจากสาเหตุอะไรบ้างคะ,"เคยเป็นค่ะ เคยอ้านเจแว่า22-23ถ้าไม่มาหรือมาไม่ปกติให้มาปรึกษาแพทย์ อายุ23จึงตัดสินใจไปตรวจดู ทีแรกให้ทานยาคุม 2,3เดือนพอหยุดยาเม็นจะมา แต่มันก็ไม่มาหมอเลยให้ซาวด์หน้าท้องเหมือน

In [89]:
all_df.comment_tokens.quantile([i/100 for i in range(1,100)]).head(20)

0.01    3.0 
0.02    4.0 
0.03    6.0 
0.04    7.0 
0.05    8.0 
0.06    9.0 
0.07    10.0
0.08    11.0
0.09    11.0
0.10    12.0
0.11    13.0
0.12    14.0
0.13    15.0
0.14    16.0
0.15    17.0
0.16    17.0
0.17    18.0
0.18    19.0
0.19    20.0
0.20    21.0
Name: comment_tokens, dtype: float64

In [91]:
all_df[all_df.comment_tokens>=8][['content_id','title','comment','comment_tokens']]\
    .sort_values('comment_tokens',ascending=True).head(30)

,content_id,title,comment,comment_tokens
2599826,36445817,คลอดลูกใหม่สามารถท้องต่อได้มั้ย,มโนป่าวเนี่ย,8
3361928,34738320,"เกมนี้ กี่โมง วันอาทิตย์ที่ 31 มกราคม 2559: Pro Bowl, MK สุกี้ และอื่นๆ อีกมาก ลุ้นกันได้ยาวๆ ทั้งวันเลยครับ",ขอบคุณครับ,8
2616112,37292656,เราเตอร์ Cisco EPC3825 สามารถจำกัดความเร็วเน็ตของแต่ละเครื่องได้ไหมครับ,ไม่ได้[URL],8
1585487,38883439,app ของ ธ.กสิกรไทย กับ ธ.กรุงเทพ ใช้ผ่านไวไฟหรือผู้ให้บริการ(ซิม) ของอเมริกา ในประเทศอเมริกา ได้หรือไม่ครับ,ขอบคุณครับ,8
3361947,34738700,| นี่ MB ทีมชาติไทยไง.. จะใครเล่า? |,สวยทั้งคู่คะ,8
694758,31569389,ข่าวดี เขตสายไหม คลองสาน พลังของประชาชนกดดัน จน กกต ยอมแล้ว,คลิปเขตสายไหม,8
1221215,37812838,ท้อง หรือ ไม่ท้อง!!,ขอบคุณค่ะ,8
3362004,34739385,นัดที่ 37 ระหว่าง บาร์ซ่า กับ เอสปันย่อน ถ้าบาร์ซ่าคะแนนนำจนนัดในบ้านแล้วชนะขาด จะฉลองแชมป์นัดนั้นไหมครับ,ขอบคุณครับ,8
2027271,34763758,EYE SHILED 21 ภาคพิเศษ เล่มนี้มันเป็นเนื้อเรื่องตอนไหนครับ,น่าจะ one shot 2 ตอน,8
2027270,34763758,EYE SHILED 21 ภาคพิเศษ เล่มนี้มันเป็นเนื้อเรื่องตอนไหนครับ,เป็น one shot ครับ,8


In [88]:
all_df[all_df.comment_tokens>=12][['content_id','title','comment','comment_tokens']]\
    .sort_values('comment_tokens',ascending=True).head(30)

,content_id,title,comment,comment_tokens
227019,33823519,php ถามเกี่ยวกับการ Key เวลาตามเงื่อนไขค่ะ,ดูฟังก์ชั่น strtotime,12
785672,31298900,มาดูครับ ว่า........ม๊อบคนดีที่ลำปางล่าสุดเย็นเมื่อวานแผ่วไปแค่ไหน,ลำปางหนาวมาก.. ฮ่าๆๆ,12
321810,31904129,Net true ล่มอยู่ใช่ไหม,ดอนเมือง ใช้ได้ อยู่,12
321809,31904129,Net true ล่มอยู่ใช่ไหม,ทุ่งครุ เน็ตใช้งานดี,12
321807,31904129,Net true ล่มอยู่ใช่ไหม,หลักสี่ โอเค อยู่ครับ,12
3172404,38483482,Galaxy s9+ ปี 2019 ยังน่าใช้อยู่ไหม ??,ก็ยังน่าใช้ครับ,12
785658,35280617,มีน้อยใช้น้อย มีเยอะใช้เยอะ คำนี้จริงไหม,96.5 ผมฟังทุกวัน,12
1737239,38440172,อยู่ย่านบางนา เลือกประกันสังคมโรงพยาบาลไหนดีคะ,รพ.สำโรงการแพทย์ค่ะ,12
4020118,34603121,หน้าจอไอโฟนแตกเปลี่ยนศูนย์หรือร้านทั่วไปดีคะ,ศูนย์ ดีกว่าครับ,12
3172486,38482340,Samsung Galaxy S9+ ได้อัพเดทเป็น android pie แล้ว ใครอัพเดทแล้วบ้างครับ,ขอ Back up ข้อมูลก่อน,12


In [90]:
all_df[all_df.comment_tokens>=21][['content_id','title','comment','comment_tokens']]\
    .sort_values('comment_tokens',ascending=True).head(30)

,content_id,title,comment,comment_tokens
2160026,34874292,อ้อม สุนิสา กับ นิโคล เทริโอ สมัยก่อนใครดังกว่ากัน?,ดังคนละยุคนะครับ...แต่ดังทั้งคู่,21
3404016,38586815,เด็ก BNK คนไหนเคยมา Live ด้วยหน้าสดบ้าง,ถ้าจะเอาสดกว่านี้ก็ปลาในน้ำละครับ,21
3815916,33088038,ลีลาวดีสีเพลิง เล่นสะกดจิตเปลี่ยนความทรงจำใหม่เลยเหรอ,โถ คุณวินสุดหล่อ ไม่น่าเล้ย,21
1489418,38728381,สงสัยครับทำไมคนสมัยก่อนห้าม เด็กๆใช้แป้งหอมตาบู,ที่รรประจำ ไม่ให้ใช้เพราะรูปคนจูบกัน,21
3179021,38989371,Review งานจับมือ จูเน่ คิดว่าพ่อกับแม่รักลูกเท่ากันมั้ย,ถามเเบบนี้นึกว่างานสัมภาษณ์นิตยสาร,21
2608760,37381494,ใครเคยเล่น toreba บ้างคะ:)),ลงทะเบียนตรง Crad number ทำยังไงหรอค่ะ,21
2608764,37381549,ใช้ สปีดเทส เทสเน็ตบ้าน จากในคอมกับมือถือ ทำไมได้ความเร็วไม่เท่ากันครับ,samsung galaxy core 2แล้วอันไหนจริงครับ,21
4338997,35870568,อยากจะกินลาบเลือดที่คั่วสุก จะต้องสั่งเขาว่าอย่างไรครับ,ลาบคั่วแบบทางเหนือพอใกล้เคียงไหมครับ,21
4829696,30083184,อาร์เซน่อล เชลซี แมนซิตี้ ลิเวอร์พูล ทีมไหนมีโอกาสแชมป์ว่าวมากสุด,พวกโยงมา United นี่มีตาแต่หามีแววไม่,21
2724757,37113218,TRUE หมายถึง อะไร ตามความรู้สึกของนักลงทุน,สิ้นปี น่ามีเซอร์ไพรส์ นะตัวนี้,21


In [92]:
all_df.shape, all_df.content_id.nunique()

((4868579, 29), 1206973)

In [134]:
#min tok 12 (10th percentile) to filter out ขอบคุณ
#max tok 512 (97th percentile) to filter out too detailed answers
all_df = all_df[(all_df.comment_tokens>=12)&(all_df.comment_tokens<=512)]
all_df.shape, all_df.content_id.nunique()

((4301481, 30), 1196548)

In [137]:
all_df[['title','body','comment','comment_tokens']].sort_values('comment_tokens')

,title,body,comment,comment_tokens
3516102,พื้นปาร์เก้ควรเลือกใช้เฟอร์นิเจอร์โทนสีไหน ?,พอดีที่บ้านเพิ่งปูพื้นปาเก้อะครับอยากทราบว่าควรเลือกเฟอนิเจอร์แบบไหนให้เข้ากับพื้นห้องดีครับ,ได้หมดชอบแบบไหนมากกว่า,12
3802825,คาดเชือก ใครรำคาญ ละเมียน ละไม และเด็กสตางค์บ้าง,คือดูมาหลายตอนแล้ว แต่ที่รู้สึกรำคาญคือละไม ที่แสดงโดยจั๊กจั่น ทั้งเรื่อง ได้ยินแต่เสียงแหลม แว๊ดๆ ฟังไม่ค่อยรู้เรื่องยิ่งตอนแรก โวยวาย ตลอดเวลา จั๊กจั่นควรฝึกพูด ปรับน้ำเสียงให้ดูหนักแน่นมากกว่านี้เข้าใจนะครับ เป็นเป็นคนเสียงเล็ก แต่ตอนโวยวาย บางทีนี้ฟังไม่รู้เรื่องเลยครับละเมียน น้องโบว์คนน่ารัก ชอบด่าชอบว่า บุญสม ใจร้อน วู่วาม แต่ตัวเอง ไม่ก็ได้น้อยหน้าไปกว่ากันหัวรั้น ดื้อคิดว่าตัวเองเก่ง ชอบด่าบุญสม ปลัดขี้คุก แบบน่าหมั่นไส้ แต่ พอบุญสมโต้กลับกลับพูดว่า รังแกผู้หญิง ไม่แมน ใช้คำว่ารังแกผู้หญิง มาเป็นเกราะกำบังตัวเด็กสตางค์ ดื้อเอาแต่ใจ พูดจาไม่คิด ทำผิดแล้ว ละไม ไม่ลงโทษ กลับไปโทษคนใช้อย่างตอนที่หนีลงเรือไปหาตามหาพยานด้วยกัน ละไมไม่เด็ดขาดกับเด็กสตางค์ทำให้เด็กได้ใจ อยากพูดอะไร ทำอะไรก็ได้ แถมละไมยังกลับมาตำหนิคนรับใช้ที่ดูแลอีกละครแบบนี้เด็กๆดูเยอะนะครับ น่าจะมีบทที่ลงโทษ หรือว่าตำหนิ ตักเตือนให้เห็นหน่อยวันนี้สตางค์โดนทองดีจัดการ ไม่ได้รู้สึกสงสารเท่าไหร่เลย (สงสัยผมจะไม่ชอบเอามาก)อันนี้เป็นความคิดเห็นผมนะครับ ใครคิดยังไงบ้างครับ แสดงความคิดเห้นกัน แต่อย่าด่าผมนะครับ ^^แก้ไขชื่อตัวละครับ,สงสารสตางค์มากกว่าคะ,12
3802844,ลมซ่อนรักถ่ายทำที่ไหนคะ?,โลเคชั่นสวยมาก อยากทราบว่าถ่ายทำที่ไหนค่ะ ขอบคุณมากๆค่ะ,ฟาร์ม จิม ทอมสัน,12
3802910,โดนผู้หญิงตบบ่อยมาก ถ้าจะจูบกลับแบบในละคร จะผิดไหม??,คือมีเรื่องกับผู้หญิงบ่อยมากๆครับโดนตบเกือบทุกวัน แต่ก็ไม่ได้ทำอะไร เพราะตัวเองเป็นผู้ชายแมนๆอย่างมากก็แค่ด่า ไม่ใช้กำลังทีนี้คิดว่า จะจูบกลับบ้างแบบในละครแนวตบจูบ เพราะผู้หญิงบางคนริมฝีปากบาง น่าจูบมากๆ พูดแล้วเปรี้ยวปาก !,ดิชั้นพร้อมค่ะ!,12
3803001,ถ้าวันนี้ เพลิงพระนาง !! คุณกบให้สัมภาษณ์ว่า รับเล่บท เศกขระเทวี แน่นอน จะเกิดอะไรขึ้น ??????,เราจะกรี๊ดดดดดเลยมาประชันกับ ซุปตาร์ตัวแม่ อย่างอั้ม ด้วยกบ อั้ม นี่ถ้ามีนุ่นด้วยคงกรี๊ดดดดดดดหนักๆสุดๆแต่เราว่ากบไม่น่าจะเล่นนะ เพราะยังต้องเลี้ยงน้องอีก ไหนจะน้ำหนักที่ยังไม่ลดวันนี้น่าจะมีสัมภาษณ์นะ รอลุ้น,รำค่ะเพราะเราบนไว้,12
...,...,...,...,...
1480254,เเถวขอนแก่น มีวัดไหนรับลงนะหน้าทอง สาริกาลิ้นทองเมตตามหานิยมคนรักใครเอ็นดูมั้ยครับ,ผมอยากลงนะน่าน่าทอง พวกสาริกาลิ้นทอง เมตตามหานิยม คนรักใคร่เอ็นดู ขอเป็นสายพุทธคุณนะครับเเถวจังหวัดขอนเเก่นที่ไหนครับ ค่าครูเท่าไหร่ ถ้าจะไปต้องไปวันไหนโลกสวย เอาวิทยาศาสตร์ เอาหลักคำสอนศาสนามาอ้าง ไม่ดีอย่างโน้นไม่ดีอย่างนี้ ไม่ต้องมาเม้นนะความเชื่อส่วนบุคคล เเค่ยึดมั่นในการทำดี ไม่ทำให้ใครเดือดร้อนก็พอ,ระหว่างที่ยังไม่ได้ที่ลงนะคะขออนุญาตแชร์กันมีคนรู้จักคนหนึ่ง​พูดอะไร​ใครก็ฟัง​เป็นที่ไหน​ใครก็รักใค่ ได้ความเอ็นดูจากผู้หลักผู้ใหญ่เสมอๆเคยมีหลายๆคนบอกว่า​ โอย.... เค้ามีสาริกาลิ้นทองแล้วยิ่งเรื่องขอความช่วยเหลือจากใครไม่เคยพลาด​ได้รับความช่วยเหลือตลอดวันหนึ่ง​มีโอกาสจับเข่าคุยเลยถามเรื่องที่ได้ยินมา ว่าไปลง​นะหน้าทอง​หรือได้สาริกาจากไหนมาเจ้าตัวบอกว่า​ ไม่เคยเลยแต่ที่เค้าทำเป็นประจำหลักๆเลย​คือสวดมนต์​แผ่เมตตา​ ระลึกถึงบุญ​คุณ​พ่อ​แม่​ ครู​บา​อาจารย์​และเทวดาที่คุ้มครองถัดมา​คือ​ พูดเพราะ​ กับทุกคน(ทุกคนจริงค่ะ​ จะเป็นใคร​รปภ.​คนขายของ​ แม้กระทั่งเด็กขายพวงมาลัย​เค้าก็พูดเพราะด้วยตลอด)​ต่อมา​คือ​มีสัจจะ​ พูดอะไร​ รักษาคำพูดตัวเองตลอด​ แต่ถ้าเกิดเหตุสุดวิสัย​ทำให้พลาดไป​ก็รีบแจ้ง​ รีบขอโทษสุดท้าย​คือ​ไม่พูดถึงใครในแง่ร้ายลองดูนะคะ​ กำลังพยายามทำอยู่เหมือนกัน,512
3205756,***********มาดูเสือใต้ ทีมที่เค้าบอกว่าเป็นจอมดูดนักเตะ แต่ละคนมาจากไหนกันมั่ง************,นายประตูมานูเอล นอยเออร์-ชาลเก้ อันนี้คงเข้าข่ายดูดทอม สตาร์เค่-ฮอฟเฟ่นไฮม์ อ่ะ ดูดจากทีมหมู่บ้านมั๊งลูคัส เรเดอร์-เด็กปั้นกองหลังดันเต้-กลัดบัค อ่ะ ดูดมั๊งฟาน บุยเต็น-ฮัมบูร์ก นี่ก็ดูดเจโรม บัวเต็ง-แมน ซิตี้ ดูดข้ามลีกกันเลยราฟินญ่า - เจนัว ก่อนอยู่เจนัวเคยอยู่ชาลเก้ ถือว่าดูดดาวิด อลาบา-เด็กปั้นดีเอโก้ คอนเตนโต้-เด็กปั้นฟิลิปป์ ลาห์ม-เด็กปั้นโฮลเกอร์ บาดชตูเบอร์-เด็กปั้นกองกลางฆาร์บี มาร์ติเนซ-บิลเบา ดูดข้ามลีกอีกแระติอาโก้ อัลคันตาร่า-บาร์ซ่า ต่างดาวยังไม่เว้นบาสเตียน ชไวน์สไตเกอร์-เด็กปั้นโทนี่ โครส-เด็กปั้น(จริงๆย้ายจากฮันซ่า รอสต็อคตอนอายุ 16 ถือว่าดูดมั๊ยเนี่ย)โทมัส มุลเลอร์-เด็กปั้นมาริโอ เกิทเซ่-ดอร์ทมุนด์ อันนี้ดูดของแท้ปิแอร์ เอมิล ฮอยเบิร์ก-เด็กปั

## Pick Sequence Length for Prompts

In [187]:
all_df['prompt_tokens'] = all_df['title_tokens'] + all_df['body_tokens']

In [154]:
all_df.title_tokens.describe().round(3)

count    4301481.000
mean     34.777     
std      14.499     
min      3.000      
25%      24.000     
50%      32.000     
75%      43.000     
max      151.000    
Name: title_tokens, dtype: float64

In [155]:
all_df.title_tokens.quantile([i/100 for i in range(1,100)]).head(20)

0.01    11.0
0.02    13.0
0.03    14.0
0.04    15.0
0.05    16.0
0.06    16.0
0.07    17.0
0.08    17.0
0.09    18.0
0.10    18.0
0.11    19.0
0.12    19.0
0.13    20.0
0.14    20.0
0.15    20.0
0.16    21.0
0.17    21.0
0.18    22.0
0.19    22.0
0.20    22.0
Name: title_tokens, dtype: float64

In [160]:
all_df[all_df.title_tokens<=16][['content_id','title','body','title_tokens','prompt_tokens']]\
    .sort_values('title_tokens',ascending=True).head(30)

,content_id,title,body,title_tokens,prompt_tokens
826323,30899596,ทำไม?,1. ทำไม ปชป. ไม่คิดปรับเปลี่ยนนโยบาย...เพื่อที่จะเปลี่ยนใจคนเสื้อแดงมาเป็นฝ่ายสนับสนุนตน...2. ทำไม ปชป. ไม่เปลี่ยนรูปแบบจากการพูด / ใช้วาทะกรรม...มาเป็นการเริ่มต้นทำงานอย่างจริงจัง..หาข้อมูลหลักฐานมาเผด็จศึกรัฐบาล3. ทำไม สมาชิกพรรค ปชป. จึงต้องเดินตามรอย นายชวน..ทุกอย่างก้าว ทั้งคำพูด ลีลา ...ฤา..พรรคนี้เป็นของนายชวน....4. ทำไม ปชป. ก่อตั้งเป็นพรรคการเมืองมาร่วม 60 ปี..ขึ้นบริหารประเทศมาหลายรอบ..ประเทศไทยก้าวไปได้แค่เนี้ย......5. ทำไม ..นายชวน..ไม่ยุติบทบาททางการเมือง..อายุก็มากแล้ว...หรือมีอะไรแอบแฝง6. ทำไม..ปชป..จึงรักสถาบันมากกว่าคนกลุ่มอื่น...7. ทำไม..ปชป. หายใจเข้าออกเป็นทักษิณ...ทั้งๆที่คนส่วนใหญ่เขาก้าวข้ามทักษิณไปหมดแล้ว8. ทำไม..อะปิสิด..นายชวน..ไม่ยอมเข้าร่วมเวทีปฏิรูป...เห็นค้านตรงใหนก็เสนอในเวทีสิ....9. ทำไม..คนเก่ง..คนอื่นในพรรค ปชป.จึงไม่มีปากเสียง....คุณศุภชัย..ดร.สุรินทร์10. ทุกอย่าง...ทำไม...ก็เพราะพรรค.ปชป..เป็นพรรคการเมืองที่ห่วยแตก..ก็เท่านั้น..ใช่ใหม...อ้ายพรรคสันขวาน,3,534
2713903,36264835,ทำไม?,ทำไมแม่ต้องด่าเราเวลาเราติดเกมเกมนึงแล้วนั่งเล่นทั้งวัน?,3,37
1035506,32189263,ใครเคย,เรื่องมีอยู่ว่า เมื่อวันที่12 มีเพื่อนทักแชทมา บอกว่าไปเจอรูปเราอยู่เฟสบุ๊คคนนึง แล้วก็บรรยายภาพว่า หมอเล็กคือคนแรกและคนสุดท้ายที่ข้าวจะรักและจะใช้ชีวิตอยู่ด้วย แทรกภาพไม่ได้ เด่วมาต่อครับ,3,119
1035507,32189263,ใครเคย,เรื่องมีอยู่ว่า เมื่อวันที่12 มีเพื่อนทักแชทมา บอกว่าไปเจอรูปเราอยู่เฟสบุ๊คคนนึง แล้วก็บรรยายภาพว่า หมอเล็กคือคนแรกและคนสุดท้ายที่ข้าวจะรักและจะใช้ชีวิตอยู่ด้วย แทรกภาพไม่ได้ เด่วมาต่อครับ,3,119
1796568,38913228,ทำไม!!,เวลาเเฟนเก่าเรา:smiling_face_with_heart-eyes:มันทะเลาะกับแฟน...เเล้วชอบทักมาหาเราเหมือนกับมาให้ความหวังเรา....ทั้งๆที่มันก็รู้ว่าเรายังคิดถึงมันอยู่..เหมือนมาให้ความหวัง:face_with_raised_eyebrow::face_with_raised_eyebrow:เเต่มันก็กับไปดีกันเหมือนเดิมมีใครเคยเจอบ้าง....,3,157
759156,35526347,ทำไม?,ทำไมเมื่อเราทำผิดอะไรสักอย่าง ไม่ว่าเรื่องเล็ก เรื่องใหญ่ ต้องโดนคนอื่นตำหนิด้วยค่ะ ไม่ใช้ไม่ยอมรับผิดนะ แต่สงสัยทำไมต้องทำให้รู้สึกแย่ คนที่ตำหนิเป็นคนดีขนาดไม่เคยทำอะไรผิดเลยหรอค่ะ,3,116
759155,35526347,ทำไม?,ทำไมเมื่อเราทำผิดอะไรสักอย่าง ไม่ว่าเรื่องเล็ก เรื่องใหญ่ ต้องโดนคนอื่นตำหนิด้วยค่ะ ไม่ใช้ไม่ยอมรับผิดนะ แต่สงสัยทำไมต้องทำให้รู้สึกแย่ คนที่ตำหนิเป็นคนดีขนาดไม่เคยทำอะไรผิดเลยหรอค่ะ,3,116
759154,35526347,ทำไม?,ทำไมเมื่อเราทำผิดอะไรสักอย่าง ไม่ว่าเรื่องเล็ก เรื่องใหญ่ ต้องโดนคนอื่นตำหนิด้วยค่ะ ไม่ใช้ไม่ยอมรับผิดนะ แต่สงสัยทำไมต้องทำให้รู้สึกแย่ คนที่ตำหนิเป็นคนดีขนาดไม่เคยทำอะไรผิดเลยหรอค่ะ,3,116
1796567,38913228,ทำไม!!,เวลาเเฟนเก่าเรา:smiling_face_with_heart-eyes:มันทะเลาะกับแฟน...เเล้วชอบทักมาหาเราเหมือนกับมาให้ความหวังเรา....ทั้งๆที่มันก็รู้ว่าเรายังคิดถึงมันอยู่..เหมือนมาให้ความหวัง:face_with_raised_eyebrow::face_with_raised_eyebrow:เเต่มันก็กับไปดีกันเหมือนเดิมมีใครเคยเจอบ้าง....,3,157
3356393,34668471,ทำไม?,ดูบันทึกการแข่งขันที่ทีมชาติไทยแข่งขันในหลายๆเกม เกิดข้อสงสัยมากมายกับการเล่นของทีมชาติไทย จึงได้มาตั้งกระทู้ถาม+สนทนากับคนในนี้1ทำไมวรรนา บัวแก้ว ถึงไม่รับบอลแรกและขุดเองทั้งหมดเลย ทำไมถึงต้องเปลี่ยนกับปิยนุด แป้นน้อย2ทำไมอรอุมา สิทธิรักษ์ไม่จั้มโฟล์ตเสิร์ฟเหมือนคนอื่นๆ ทำไมต้องจั้มพาวเวอร์ฟูลเสิร์ฟ3ทำไมไม่เคยเห็นมัลลิกา กันทอง หยอดเลย ส่วนมากจะตีๆทั้งนั้นทีมอื่นจะจับทางได้ง่ายไม่ค่อยหลากหลายในเรื่องการตี4ทำไมในช่วงหลังๆวิลาวรรณ อภิญญาพงษ์ ถึงต้องสำรอง เห็นตัวผู้เล่นเป็น อรอุมา สิทธิรักษ์ อัจฉราพร คงยศ มัลลิกา กันทองถ้าจะเอารุกวิลาวรรณ อภิญญาพงษ์ ก็รุกได้ตอนชิงแชมป์เอชเชียสโมสรที่เวียดนาม และการรับก็ทำได้ดีที่สุด,3,387


In [161]:
all_df[all_df.title_tokens>=16][['content_id','title','body','title_tokens','prompt_tokens']]\
    .sort_values('title_tokens',ascending=True).head(30)

,content_id,title,body,title_tokens,prompt_tokens
1450878,38802112,ทำไม r15 ไม่ทำเครื่อง dohc มาขาย,ถ้าr15 ทำเครื่อง dohc 155 cc มี vva พี่ๆคิดว่าจะเป็นไงบ้างครัฟ จะดีกว่าเครื่องปัจจุบันไหม,16,71
4069084,35153388,มีวิธีกู้ภาพจาก icloud ไหม?,เอาโทรศัพท์ไปซ่อม แล้วเอาอีกเครื่องมาใช้แต่พอใส่ icloud แล้วรูปไม่เข้า แต่ของแม่ใส่ icloud แล้วรูปก็มาครบอยากจะทราบว่าพอจะมีวิธีกู้รูปได้ไหมแม้จะได้ไม่หมดก็ไม่เป็นไรขอบคุณทุกคอมเม้นล่วงหน้าค่ะ,16,142
4069083,35153388,มีวิธีกู้ภาพจาก icloud ไหม?,เอาโทรศัพท์ไปซ่อม แล้วเอาอีกเครื่องมาใช้แต่พอใส่ icloud แล้วรูปไม่เข้า แต่ของแม่ใส่ icloud แล้วรูปก็มาครบอยากจะทราบว่าพอจะมีวิธีกู้รูปได้ไหมแม้จะได้ไม่หมดก็ไม่เป็นไรขอบคุณทุกคอมเม้นล่วงหน้าค่ะ,16,142
4663498,31909164,จอ nexus5 ทำไมกินแบตจังครับ,ทั้งที่เปิดความสว่างหน้าจอแค่นิดเดียวเอง,16,40
4663497,31909164,จอ nexus5 ทำไมกินแบตจังครับ,ทั้งที่เปิดความสว่างหน้าจอแค่นิดเดียวเอง,16,40
4663496,31909164,จอ nexus5 ทำไมกินแบตจังครับ,ทั้งที่เปิดความสว่างหน้าจอแค่นิดเดียวเอง,16,40
4663495,31909164,จอ nexus5 ทำไมกินแบตจังครับ,ทั้งที่เปิดความสว่างหน้าจอแค่นิดเดียวเอง,16,40
4069082,35153388,มีวิธีกู้ภาพจาก icloud ไหม?,เอาโทรศัพท์ไปซ่อม แล้วเอาอีกเครื่องมาใช้แต่พอใส่ icloud แล้วรูปไม่เข้า แต่ของแม่ใส่ icloud แล้วรูปก็มาครบอยากจะทราบว่าพอจะมีวิธีกู้รูปได้ไหมแม้จะได้ไม่หมดก็ไม่เป็นไรขอบคุณทุกคอมเม้นล่วงหน้าค่ะ,16,142
4069015,35152600,มีใครเคยโดน AIS ทำแบบนี้บ้าง,"เราเปลี่ยนจากระบบเติมเงินมาเป็นรายเดือน ซึ่งในระบบเติมเงินมียอดเงินคงเหลือและได้ไปติดต่อ AIS ทำเรื่องขอคืนเงินเจ้าหน้าที่แจ้งว่าไม่สามารถทำคืนเงินสดได้แต่จะทำคืนเงินเข้ามาในระบบรายเดือนรายเดือนเราซื้อเพคเกจรายเดือน 599 ได้โทรฟรี 650 และจ่ายล่วงหน้า 3800 ปกติแต่ละเดือนก็ใช้โทรไม่เคยหมดอยู่แล้วตั้ง 650 นาทีแต่เราได้ทวงถามกับ AIS ซี่ง AIS แจ้งว่าตามนี้ค่ะ""สำหรับกรณีที่มียอดเงินคงเหลือในระบบเติมเงิน เมื่อมีการเปลี่ยนมาเป็นระบบรายเดือนแล้ว จะมอบเป็นสิทธิ์โทรฟรีให้ลูกค้าอัตโนมัติครับโดยระบบมอบเป็น รับสิทธิ์โทรฟรีทั่วไทย 750 บาท นาน 12 เดือน""AIS เคยถามลูกค้ามั้ยว่าลูกค้าอยากได้สิทธิ์โทรฟรีหรือป่าว แต่ขอโทษนะค่ะมันไม่ใช้สิทธิ์โทรฟรีแต่มันเป็นเงินของลูกค้าที่โอนมาจาก one2callเรายังยืนกับ AIS ว่าเราต้องการเงินคืนเพื่อมาหักเป็นค่าใช้จ่ายรายเดือน",16,455
4069014,35152600,มีใครเคยโดน AIS ทำแบบนี้บ้าง,"เราเปลี่ยนจากระบบเติมเงินมาเป็นรายเดือน ซึ่งในระบบเติมเงินมียอดเงินคงเหลือและได้ไปติดต่อ AIS ทำเรื่องขอคืนเงินเจ้าหน้าที่แจ้งว่าไม่สามารถทำคืนเงินสดได้แต่จะทำคืนเงินเข้ามาในระบบรายเดือนรายเดือนเราซื้อเพคเกจรายเดือน 599 ได้โทรฟรี 650 และจ่ายล่วงหน้า 3800 ปกติแต่ละเดือนก็ใช้โทรไม่เคยหมดอยู่แล้วตั้ง 650 นาทีแต่เราได้ทวงถามกับ AIS ซี่ง AIS แจ้งว่าตามนี้ค่ะ""สำหรับกรณีที่มียอดเงินคงเหลือในระบบเติมเงิน เมื่อมีการเปลี่ยนมาเป็นระบบรายเดือนแล้ว จะมอบเป็นสิทธิ์โทรฟรีให้ลูกค้าอัตโนมัติครับโดยระบบมอบเป็น รับสิทธิ์โทรฟรีทั่วไทย 750 บาท นาน 12 เดือน""AIS เคยถามลูกค้ามั้ยว่าลูกค้าอยากได้สิทธิ์โทรฟรีหรือป่าว แต่ขอโทษนะค่ะมันไม่ใช้สิทธิ์โทรฟรีแต่มันเป็นเงินของลูกค้าที่โอนมาจาก one2callเรายังยืนกับ AIS ว่าเราต้องการเงินคืนเพื่อมาหักเป็นค่าใช้จ่ายรายเดือน",16,455


In [163]:
all_df.title_tokens.quantile([i/100 for i in range(1,100)]).tail(20)

0.80    46.0
0.81    47.0
0.82    48.0
0.83    48.0
0.84    49.0
0.85    50.0
0.86    51.0
0.87    52.0
0.88    53.0
0.89    54.0
0.90    56.0
0.91    57.0
0.92    58.0
0.93    60.0
0.94    62.0
0.95    64.0
0.96    66.0
0.97    68.0
0.98    71.0
0.99    75.0
Name: title_tokens, dtype: float64

In [165]:
all_df[all_df.title_tokens<=75][['content_id','title','body','title_tokens','prompt_tokens']]\
    .sort_values('title_tokens',ascending=False).head(30)

,content_id,title,body,title_tokens,prompt_tokens
514601,37353551,อยู่หอในมหาลัยแถวลาดกระบัง ได้ยินเสียงกลองดังมากมาจากในมหาลัยนี่เข้าข่ายรบกวนผู้อื่นยามวิกาลไหมครับ ได้ยินมา 2-3 วันแล้ว,คือ จขกท. เป็นนักศึกษาของสถาบันนั้นเหมือนกันและสถานที่ตั้งของหอพักก็อยู่ห่างจากมหาลัยมากพอสมควร แต่ก็ได้รับความเดือดร้อนมากเนื่องจากจำเป็นต้องทำการบ้านในเวลากลางคืน ทั้งง่วงและไม่มีสมาธิในการทำงานจะนอนก็นอนไม่หลับเพราะได้ยินเสียงเจ้ากรรมนี่ลอยเข้ามาทุกที บางวันเสียงดังยาวนานตั้งแต่ช่วง 5 ทุ่มไปจนถึงตี 4 ของอีกวันรู้สึกแย่มากจริงๆเพราะว่านอกจากจะไม่ได้นอนเพราะทำงานแล้วยังคิดงานไม่ออกอีกด้วยต้องใส่หูฟังหรือปิดหน้าต่างตลอด เราสามารถร้องเรียนหรือแจ้งไปช่องทางไหนได้บ้าง รบกวนขอความคิดเห็นและข้อเสนอแนะจากผู้ที่เคยประสบปัญหาเดียวกันหรือเคยประสานงานเรื่องพวกนี้ด้วยครับขอบคุณครับปล.กระทู้นี้เป็นกระทู้แรกพิมพ์ข้อความไหนไม่เหมาะสมหรือไม่ถูกต้องก็ขออภัยมา ณ ที่นี้ด้วยครับ,75,474
1454076,38751003,ทำไมรถเก๋งบางคันถึงกลับรถให้อยู่ในเลนตัวเองไม่ได้ ต้องยื่นไปเลน 2 ทั้งๆ ที่มีเลนกลับรถหรือมีแบริเออร์กั้นให้ 1 เลนแล้ว,ทางกลับรถบางที่ที่ถนนกว้าง มีหลายเลน จะทำเลนกลับรถให้ คือกลับรถไปอีกฝั่งแล้ว มีเลนพักให้ 1 เลน หรือบางที่มีแบริเออร์กั้นให้ด้วย ทำให้ยังไงรถอีกฝั่งก็ไม่มีทางขับมาเลนนี้ได้อยู่แล้ว แต่ก็จะเห็นรถเก๋ง (เฉพาะรถเก๋งไม่นับกระบะ) บางคันไม่ว่าเล็กหรือใหญ่ ยังไม่กล้ากลับรถ หรือชักช้า บางคันพอเห็นกลับรถแล้วก็เห็นว่าอยู่ในเลนตัวเองไม่ได้ทำให้บางทีเกิดการสะสมของรถที่จะยูเทิร์นทำให้แถวยาว ก็เลยอยากจะรู้ว่าปัญหานี้มันเป็นที่ฝีมือคนขับ รุ่นรถ หรือทางกลับรถกันแน่,75,360
464009,35037908,(4600&2001)วันนี้วันครอบครัว อยู่พร้อมหน้ากันอบอุ่นสุขใจ สงกรานต์ปีนี้ไม่เหงาแต่ก็มีคนที่อยากให้เงียบเหงา จริงไหมๆ ?,คนขี้เหงา วันไหนๆก็เหงาได้ไปเที่ยวมาบ้านพักดีๆโดนเลือกจองไปหมดเหลือแต่เก่าๆที่ต้องปรับปรุงเพราะคนไม่กล้าอยู่ 555ไปห้างคนก็เยอะพอดูได้ เด็กๆคึกคักเล่นสวนน้ำสนุก ไม่เหงาเลยมีเวทีดนตรีและกิจกรรมให้ดูไม่เบื่อเล่นน้ำก็ใช้น้ำไม่เปลืองพอให้เปียกเย็นชื่นใจคนที่เหงาไม่คึกคักเพราะหัวใจมันฝ่อก็ขออภัย 555ขอให้สุขใจตลอดปีละกันนะชาวราชดำเนิน,75,291
464008,35037908,(4600&2001)วันนี้วันครอบครัว อยู่พร้อมหน้ากันอบอุ่นสุขใจ สงกรานต์ปีนี้ไม่เหงาแต่ก็มีคนที่อยากให้เงียบเหงา จริงไหมๆ ?,คนขี้เหงา วันไหนๆก็เหงาได้ไปเที่ยวมาบ้านพักดีๆโดนเลือกจองไปหมดเหลือแต่เก่าๆที่ต้องปรับปรุงเพราะคนไม่กล้าอยู่ 555ไปห้างคนก็เยอะพอดูได้ เด็กๆคึกคักเล่นสวนน้ำสนุก ไม่เหงาเลยมีเวทีดนตรีและกิจกรรมให้ดูไม่เบื่อเล่นน้ำก็ใช้น้ำไม่เปลืองพอให้เปียกเย็นชื่นใจคนที่เหงาไม่คึกคักเพราะหัวใจมันฝ่อก็ขออภัย 555ขอให้สุขใจตลอดปีละกันนะชาวราชดำเนิน,75,291
464007,35037908,(4600&2001)วันนี้วันครอบครัว อยู่พร้อมหน้ากันอบอุ่นสุขใจ สงกรานต์ปีนี้ไม่เหงาแต่ก็มีคนที่อยากให้เงียบเหงา จริงไหมๆ ?,คนขี้เหงา วันไหนๆก็เหงาได้ไปเที่ยวมาบ้านพักดีๆโดนเลือกจองไปหมดเหลือแต่เก่าๆที่ต้องปรับปรุงเพราะคนไม่กล้าอยู่ 555ไปห้างคนก็เยอะพอดูได้ เด็กๆคึกคักเล่นสวนน้ำสนุก ไม่เหงาเลยมีเวทีดนตรีและกิจกรรมให้ดูไม่เบื่อเล่นน้ำก็ใช้น้ำไม่เปลืองพอให้เปียกเย็นชื่นใจคนที่เหงาไม่คึกคักเพราะหัวใจมันฝ่อก็ขออภัย 555ขอให้สุขใจตลอดปีละกันนะชาวราชดำเนิน,75,291
464006,35037908,(4600&2001)วันนี้วันครอบครัว อยู่พร้อมหน้ากันอบอุ่นสุขใจ สงกรานต์ปีนี้ไม่เหงาแต่ก็มีคนที่อยากให้เงียบเหงา จริงไหมๆ ?,คนขี้เหงา วันไหนๆก็เหงาได้ไปเที่ยวมาบ้านพักดีๆโดนเลือกจองไปหมดเหลือแต่เก่าๆที่ต้องปรับปรุงเพราะคนไม่กล้าอยู่ 555ไปห้างคนก็เยอะพอดูได้ เด็กๆคึกคักเล่นสวนน้ำสนุก ไม่เหงาเลยมีเวทีดนตรีและกิจกรรมให้ดูไม่เบื่อเล่นน้ำก็ใช้น้ำไม่เปลืองพอให้เปียกเย็นชื่นใจคนที่เหงาไม่คึกคักเพราะหัวใจมันฝ่อก็ขออภัย 555ขอให้สุขใจตลอดปีละกันนะชาวราชดำเนิน,75,291
464005,35037908,(4600&2001)วันนี้วันครอบครัว อยู่พร้อมหน้ากันอบอุ่นสุขใจ สงกรานต์ปีนี้ไม่เหงาแต่ก็มีคนที่อยากให้เงียบเหงา จริงไหมๆ ?,คนขี้เหงา วันไหนๆก็เหงาได้ไปเที่ยวมาบ้านพักดีๆโดนเลือกจองไปหมดเหลือแต่เก่าๆที่ต้องปรับปรุงเพราะคนไม่กล้าอยู่ 555ไปห้างคนก็เยอะพอดูได้ เด็กๆคึกคักเล่นสวนน้ำสนุก ไม่เหงาเลยมีเวทีดนตรีและกิจกรรมให้ดูไม่เบื่อเล่นน้ำก็ใช้น้ำไม่เปลืองพอให้เปียกเย็นชื่นใจคนที่เหงาไม่คึกคักเพราะหัวใจมันฝ่อก็ขออภัย 555ขอให้สุขใจตลอดปีละกันนะชาวราชดำเนิน,75,291
3583114,37298413,ถ้าเพลิงพระนาง กับ ล่า เข้าชิงรางวัลพร้อมกัน คิดว่า อั้ม พัชราภา หรือ หมิว ลลิตา จะได้รางวัลนักแสดงนำหญิงยอดเยี่ยม,ตอนนี้ยังให้อั้มนอนมานะ,75,90
1454078,38751003,ทำไมรถเก๋งบาง

In [166]:
#min tok 16 (5th percentile) to filter out super short titles e.g. ทำไม?
#max tok 75 (99th percentile) to filter out too long titles with too many special characters/emojis
all_df = all_df[(all_df.title_tokens>=12)&(all_df.title_tokens<=75)]
all_df.shape, all_df.content_id.nunique()

((4211050, 30), 1170437)

In [168]:
all_df.prompt_tokens.describe().round(3)

count    4211050.000
mean     266.350    
std      377.158    
min      13.000     
25%      96.000     
50%      162.000    
75%      286.000    
max      9955.000   
Name: prompt_tokens, dtype: float64

In [195]:
all_df.prompt_tokens.quantile([i/100 for i in range(1,100)]).head(20)

0.01    31.0
0.02    37.0
0.03    41.0
0.04    45.0
0.05    48.0
0.06    51.0
0.07    54.0
0.08    57.0
0.09    59.0
0.10    62.0
0.11    64.0
0.12    67.0
0.13    69.0
0.14    72.0
0.15    74.0
0.16    76.0
0.17    79.0
0.18    81.0
0.19    83.0
0.20    85.0
Name: prompt_tokens, dtype: float64

In [182]:
all_df[all_df.prompt_tokens<=1024][['content_id','title','body','prompt_tokens']]\
    .sort_values('prompt_tokens',ascending=False).head(30)

,content_id,title,body,prompt_tokens
267224,33029795,"ASUS ปัญหาที่ซ่อมแล้ว แก้ไม่หาย, พนักงาน ไมาสามารถช่วยเหลืออะไรได้","ในวันที่ 07/09/2557 ผมซื้อเครื่อง Asus N550jk จาก BananaIT ซึ่งใช้มาได้ปกติ และเครื่องเกิดอาการผิดปกติขึ้นมา ใช้มาเดือนกว่า(ไม่เคยตกไม่เคยกระแทกวางใช้งานไม่เคยยกไปไหน) หน้าจอมีปัญหากระพริบและดับเอง.ผมจึงติดต่อไปทางเพจเฟสบุ๊คของ ASUS THAILAND ซึ่งสอบถามและวิธีแก้ไขเบื้องต้นซึ่งทาง u202a#u200easusu202c ไม่ตอบหรือสแดงการช่วยเหลืออะไรในเบื้องต้น. ผมจึงโทรไปสอบถามทาง call-center และสรุปว่าผมต้องส่งเครื่องแครม.ผมจึงทำการส่งเครื่องแครม แต่ว่าก็มีปัญหาเกิดขึ้น พนักงานร้านแห่งหนึ่งได้บอกว่า ถ้าอาการไม่แสดงให้เห็นจนระบุชัดแน่นอน ไม่สามารถส่งเครมได้ (ในฐานะผมผู้เป็นผู้บริโภคและเจอปัญหา ทั้งๆที่ไม่ได้ทำให้มันเกิด เจอแบบนี้เข้าไป หมดศรัทธาทั้ง bananaIT, Asus Thailand) ผมยืนอยู่ตรงนั้นชั่วโมงกว่าๆเพื่อรันเครื่องรอจนกว่าหน้าจอมันจะกระพริบและดับไปเอง พนักงานจึงยอมให้ส่งเครม ในวันที่ 18/11/2014.ในวันที่ 23/12/2014 พนักงานได้โทรมาบอกว่าเครื่องได้รับแล้ว.วันที่ 24/12/2014 ผมจึงเดินทางไปเพื่อรับเครื่องมาใช้ และเปิดทดสอบดูอาการปรากดว่ายังเป็นอยู่ ผมถามพนักงานว่าทำไมเป็นแบบนี้. พนักงานไม่สามารถตอบคำถามหรือช่วยเหลือข้อสงสัยใดๆได้ทั้งสิ้น ได้แต่บอกว่า ลองไปส่งเครมเองที่ศูนย์ ASUS. ผมคิดว่ามันเป็นหน้าที่ของผมหรอ ที่ส่งเครมไปแล้ว 1 รอบและเครื่องกับมามีปัญหาเหมือนเดิม มันเป็นหน้าที่ของผมหรอ ที่ต้องเสียค่าเดินทางหลายรอบหรือเสียเวลาของผม. ผมไม่ได้ทำอันนี้ขึ้นมาเพื่อร้องขอส่งซ่อมอีกรอบ เพราะผมมั่นใจบริการของทาง 2 บริษัทนี้แล้วว่าส่งซ่อมไปเหมือนส่งหมกไว้มากกว่า เพราะได้เครื่องกับมางานประกอบของเครื่องนั้นมีปัญหาทั้ง[URL] เม๊าส์แหวะออกมา2.ที่เสียบบูเรปิดไม่สนิท3. หน้าจอมีรอยแหว่งของงานประกอบและทีนี้ ผู้บริโภคสมควรไว้ใจงานบริการของท่านอีกมั้ยครับขอเตือนเพื่อนๆและเพจทั้งหลายว่าระวังการโฆษณาที่เกินจริงและบริการที่มีไว้เพื่อเห็นแก่เงินนะครับขอบคุณพื้นที่พันทิพมากครับ",1024
825994,34899549,Lazada ทำแบบนี้หมายความว่าอย่างไร ช่วยมาเคลียร์ด่วนด้วยครับ,"ผมเป็นคนหนึ่งที่ซื้อของจาก Lazada ค่อนข้างบ่อยมากๆ ทั้งของตนเองทั้งคนอื่นฝากซื้อ ซึ่งก็ไม่เคยคิดว่าตนเองจะต้องเป็นหนึ่งในผู้โชคร้ายจากเหตุการณ์เหล่านี้ ขอเริ่มเรื่องเลยละกัน ในวันที่ 5มีนา ผมได้สั่งสายคล้องกล้องรุ่น Artisan&Artist รุ่น 310N จาก Lazada โดยมีร้าน TK Trading เป็นผู้จัดจำหน่าย ในราคา 5,790 บาท เพื่อหวังจะเอามาใช้คู่กับเจ้า OLYMPUS OMD EM10 ตัวน้อยๆของผม โดยหลังจากสั่งซื้อเรียบร้อยโดยชำระค่าสินค้าผ่านบัตรเครดิตจากนั้นวันที่ 7มีนา ทาง Lazada ก็แจ้งยกเลิกบอกว่าไม่มีสินค้าผมจึงลอง Search สินค้าตัวนี้จาก Lazada อีกครั้งพบว่ามีสินค้าอยู่ แต่ราคาสินค้าได้ปรับขึ้นอีก 1,000 เป็น 6,790 บาท สำหรับผมราคานี้ก็ยังพอรับได้อยู่ จึงกดสั่งไปอีกรอบ (สินค้าชิ้นนี้ราคามือ1ปกติอยู่เส้นละ 8,800)และแล้วในวันนี้ 11มีนา สินค้าก็จัดส่งมาถึง ผมเลิกงานและกลับมาบ้านด้วยความรู้สึกตื่นเต้นจริงๆว่าจะได้สัมผัสกับสายคล้องกล้องที่ผลิตจากเส้นไหมญี่ปุ่นที่มีการอวดอ้างสรรพคุณจากผู้ใช้งานว่ามันเลิศเลออลังการมากๆผมค่อยๆเปิดซองสินค้าอย่างปราณีต และแล้วผมก็ต้องตะลึงกับสิ่งที่ได้เห็นเกิดอะไรขึ้น!!! ผมสั่งสายคล้องกล้องArtisan&Artist รุ่น 310Nแต่กลับส่งรุ่น108ที่วัสดุเป็นผ้ามาให้ผม ราคาจริงมันต่างกันหลายเท่ามากๆ ผมเสียความรู้สึกกับ Lazada จริงๆ เสียทั้งเวลาทั้งความรู้สึก ยังไงผมจะลองหาวิธีคืนสินค้ากับทาง Lazada ดูก่อน แต่อย่างไรก็อยากฝากเรื่องนี้ไว้เป็นอุทาหรณ์กับเพื่อนๆว่าสำหรับการสั่งซื้อสินค้าราคาแพง ในการรับสินค้ากรณีตัวเราไม่ได้รับเอง ก็ต้องกำชับกับผู้รับแทนด้วยว่าต้องเปิดดูของที่มาส่งก่อนอย่างละเอียดก่อนลงชื่อรับสินค้าด้วยครับ และสุดท้ายนี้หาก Admin Lazada ผ่านมาช่วยรับเรื่องราวนี้ไปแก้ไขปัญหาให้ด้วยครับ กรณีนี้ผมขอให้สาเหตุเกิดจากการทำงานที่ผิดพลาดในการจัดส่งสินค้าให้กับลูกค้านะครับปิดท้ายด้วยรูปเจ้ารุ่น 310N ที่ผมตั้งใจจะสั่งมาครอบครองครับขอบคุณครับ",1024
758885,31364885,คำถามเกี่ยวกับ นายกฯ มาตรา 7,"ผมมีข้อสงสัย 3 ข้อรบกวนผู้รู้ช่วยชี้แนะด้วยนะครับถ้าดูตามมาตรา 181 นายกฯรักษาการดูเหมือนจะลาออกจากตำแหน่งรักษาการไม่ได้แต่ผมสมมติว่านายกฯ เกิดทะลึ่งลาออกละกันนะครับ แล้วก็อ้างอิงไปที่ มาตรา 7 คือให้ปฏิบัติ ตามที่เคยปฏิบัติผมมีประเด็นอยู่ 2 ประเด็นครับ คือประเด็นที่ 1:ย้อนกลับไปดู 2 เหตุการณ์ที่คล้ายกันคือ1. สมัยที่ จอมพล ถนอม ลาออก และคุณสัญญา มาแทน2. สมัยที่ พลเอก สุจินดา ลาออก และคุณอานันท์ มาแทนโดยกรณีที่ 1 อาศัยอำนาจตามรัฐ

In [210]:
all_df[all_df.prompt_tokens>=48][['content_id','title','body','prompt_tokens']]\
    .drop_duplicates().sort_values('prompt_tokens').head(50)

,content_id,title,body,prompt_tokens
1919904,37415594,ครูโวคอลผู้หญิงของรายการ MIXNINE ชื่ออะไรแล้วมีวาร์ปกันมั้ยคะ,ตามหัวข้อเลยค่ะ,48
2125408,33237481,Fantastic Four 2015 ตามคอมมิคนี่จะไปรวมกับทีม X-men หรือเปล่าครับ,ถ้าไปรวมนี่ ไปรวมภาคไหนของ X-men,48
4652852,31807860,ระหว่างoppor1กับ4sอันไหนดีกว่ากันคับ,ระหว่างoppor1กับ4sอันไหนดีกว่ากันคับ,48
339362,31454510,VAIO Fit14 น่าเล่นไหมครับ จะเอามาใช้งานทั่วๆ ไปครับ,แบบว่าดีไซน์มันโดนครับผม,48
4041878,34835916,มี dtac เติมเงินจะสมัครโปรสู้สุดใจได้ไหม,หรือต้องย้ายค่ายออกแล้วย้ายเข้ามาใหม่,48
2913599,33762483,Ties เมื่อไหร่จะหลุด SP หว๋า งบก็ส่งแล้วแต่งบที่งบนี้ขาดทุน,sp นานไปแล้ว,48
4829715,30084299,"ทำไมถึงไม่ค่อยมีกรรมการหรือผู้จัดการทีม ""ผิวสี"" ในลีกดังต่างๆเลยครับ ??",ตามนั้น,48
1305275,37222387,DDD พื้นฐานดีมาก เหมือนที่โบรกเกอร์บอกไหมครับ,น่าสนใจไหม เห็นโบรกบอกจะไป 3 หลัก,48
3502851,30591562,เดือนนี้จะมีงานลดราคาเครื่องใช้ไฟฟ้าที่ไหนบ้าง,พอดีต้องซื้อเครื่องไฟฟ้าหลายชิ้นเลย,48
1912915,37482225,สรุปอูจินพูดว่าอะไร,อ่านในทวิตแล้วงงๆค่ะ น้องพูดว่าอะไรกันแน่คะ,48


In [211]:
all_df.prompt_tokens.quantile([i/100 for i in range(1,100)]).tail(20)

0.80    335.0 
0.81    347.0 
0.82    360.0 
0.83    374.0 
0.84    389.0 
0.85    406.0 
0.86    425.0 
0.87    446.0 
0.88    469.0 
0.89    496.0 
0.90    527.0 
0.91    563.0 
0.92    606.0 
0.93    656.0 
0.94    720.0 
0.95    803.0 
0.96    916.0 
0.97    1079.0
0.98    1350.0
0.99    1896.0
Name: prompt_tokens, dtype: float64

In [212]:
all_df[all_df.prompt_tokens<=1024][['content_id','title','body','prompt_tokens']]\
    .drop_duplicates().sort_values('prompt_tokens',ascending=False).head(50)

,content_id,title,body,prompt_tokens
3238827,32362126,ขอถามสมาคมเทควันโด จะรับผิดชอบอย่างไร หากนักกีฬาที่คัดไปทำให้ประเทศเสียชื่อ,เป็นที่แน่ชัดแล้วว่า สมาคมยังไม่ปิดโอกาสที่จะให้ ก้อย ติดทีมชาติไปโอลิมปิก ประกอบกับ ตัวก้อยเองและครอบครัว ก็พลิกลิ้น กลืนน้ำลาย ใช้กระแสดังกล่าวเป็นโอกาส คว้าไว้เพื่ออะไรก็แล้วแต่ ตีเนียนหวนคืนทีมชาติ ตามคำเชิญของสมาคมซะงั้น!!!เรื่องโค้ชเชกับก้อย ก็เป็นที่แน่ชัดแล้ว จากเวลาพิสูจน์คน ว่าใครโกหก ใครพูดจริง ใครดี ใครไม่ดี อันนี้ปรากฎภาพจริงเสียงจริง มาเยอะแล้ว ขอข้ามไปแต่ขอถามชัด ๆ ซ้ำ ๆ อีกครั้งว่า หากเกิดเหตุการณ์เดิมอีก นักกีฬาที่คัดไปเป็นตัวแทนทีมชาติ เป็นตัวแทนของคนทั้งชาติ ไปในนามประเทศไทย แบกธงชาติบนหน้าอก เอาเอกลักษณ์และวัฒนธรรมไทยไปกับตัวด้วย ไปโชว์ด้วย หรืออีกนัยหนึ่ง คือคนต่างชาติเค้าก็รู้ว่านี่นะ ตัวแทนคนไทย แต่กลับไปแสดงความไม่มีน้ำใจเป็นนักกีฬา หรือไปแสดงอาการงอแง ไร้ระเบียบวินัยอีก สร้างภาพลบ ชื่อเสีย ให้กับคนไทย ในนามตัวแทนคนไทยทั้งชาติ บนเวทีระดับโลกอีกถ้าเกิดเหตุการณ์อย่างนี้ สมาคมเทควันโด เตรียมการ หรือมีมาตรการอะไร ที่จะรับผิดชอบ หรือป้องกันมิให้เกิดเหตุการณ์แบบนี้อีกอย่าลืมนะครับ การที่คุณหรือใคร สมัครใจ ที่จะรับใช้ชาติ เป็นตัวแทนของคนทั้งชาติ คุณจะต้องแบกเอาทั้งความหวัง ความรับผิดชอบ เอาหน้าตาประเทศชาติ ไปกับคุณด้วย ตรงนี้ จะรับไหวมั้ย อีกอย่าง ไม่มีใครบังคับให้คุณเป็นตัวแทนของคนทั้งชาตินะครับ ถ้าคุณไม่พร้อม ไม่มีเลยซักคนที่จะบังคับ หน้าที่ตรงนี้ ใหญ่หลวงนักแล้วกับพฤติกรรมเดิม ๆ ที่ผ่านมาจนถึงปัจจุบัน ก้อยเอง ยังไม่เคย ไม่คิดที่จะแสดงอาการรับผิดชอบกับเหตุการณ์ที่ตนได้กระทำให้ประเทศ และเพื่อนร่วมชาติเสียหายเลย ไม่เคยขอโทษ หรือมีความจริงใจที่จะปรับเปลี่ยนพฤติกรรมที่ไม่ดีของตน ในเมื่ออยากจะกลับมาคว้าโอกาสที่จะทำให้ตนและคนของตนอยู่รอด มีที่ยืนต่อในสังคม................. แล้วอย่าบอกนะครับว่าไม่มีทางที่จะเกิดเหตุการณ์เดิมซ้ำ ๆ อีก อันนี้ไม่ขอเชื่อสมาคมเทควันโดช่วยตอบหน่อยครับ??,1024
1301818,37279943,ทำยังไงให้แม่อยากลงทุนกองทุนรวมหรือหุ้นบ้าง,คือแม่ดิชั้นเป็นราชการครูค่ะ กำลังจะเกษียณอีกไม่กี่เดือนข้างหน้า เป็นครูซี 8 เงินเดือนหลักหลายหมื่น ถ้าเค้าเกษียณเงินเดือนจะได้อยู่ประมานสี่หมื่นไปจนตายค่ะ แม่เป็นคน conservative หรืออนุรักษ์นิยมมากเค้าโตมากับการหยอดเงินในกระปุกและแน่นอน เค้าก็สอนดิขั้นมาเช่นนั้นตลอดค่ะแม่ไม่มีหนี้สิน ไม่มีอะไรที่ต้องทุกข์ และแม่ก็ไม่มีความรู้เรื่องการเงินเช่นเดียวกัน รู้แต่ว่าต้องออมในสหกรณ์ราชการเพื่อได้กินปันผลเยอะๆ 6-8% ต่อปี แค่นั้นเองค่ะแต่ตอนนี้ดิชั้นอยู่ กทม แม่อยู่บ้าน ตจว. ดิชั้นได้เรียนรู้เรื่องการลงทุนต่างๆ เลยเอากลับไปอธิบายให้แม่ฟังว่าเงินที่เก็บในสหกรณ์หลายล้านบาทนั้น มันจะโดนเงินเฟ้อปีละ 3% ถึงแม้ปันผลจะดี อยากให้แม่เอาเงินเดือนบางส่วนหลังเกษียณที่ได้ทุกเดือนไปลงทุนในหุ้น หรือกองทุนหุ้นบ้างก็ได้ เพราะดิชั้นเชื่อว่าแม่ดิชั้นไม่ตายง่ายๆค่ะ อยู่ไปอีกหลายสิบปีแน่นอนเพราะแกแข็งแรงมาก (ดูจากยาย ตอนนี้จะ 90 ละค่ะยังแข็งแรงมากอยู่เลย)แต่แม่ตอบมาว่า แม่ไม่มีความรู้เรื่องนี้ ไม่เอาอย่างเดียว แกกลัวถูกโกงบ้าง กลัวความเสี่ยงไปหมดแม่บอกว่า รุ่นแม่นี้มันไม่ใช่เรื่องต้องมาคิดอะไรพวกนี้แล้ว คิดอย่างเดียว ขออยู่อย่างมีความสุข เพราะลำบากมานาน แค่นั้นทุกวันนี้แม่ก็ยังคิดไม่ออก ว่าถ้าเกษียณแล้วแกจะเอาเวลาว่างๆมากมายไปทำอะไร อาจลงมาอยู่บ้านพี่ที่ กทมแล้วช่วยเลี้ยงหลานซึ่งตอนนี้ 3 ขวบ แต่แกก็เคยมาแล้วนะคะ มาได้แค่ 7 วัน จะกลับบ้าน ตจว. อย่างเดียวเพราะแกเบื่อ หลานก็ไปโรงเรียน แกไม่มีอะไรทำ และที่ กทม แกก็ไม่รุ้จักใคร มันไม่เหมือนบ้านที่ ตจว. รู้จักกันทั้งหมู่บ้านไปเที่ยวบ้านคนนุ้นคนนี่ได้ ไม่เบื่อ ไม่เหงาดิชั้นจะทำยังไงดีคะ อธิบายไปเค้าก็ไม่เข้าใจ จะแอบพาไปสัมมนา เค้าก็กลัวว่าไปขายตรงรึป่าวอีก 55ดิชั้นแค่หวังดีกับแม่ อยากให้แม่มีความรู้เรื่องเงินบ้าง อย่างน้อยๆรู้แค่ว่าความน่ากลัวของเงินเฟ้อมันเป็นอย่างไรแค่นั้นก็ดีละค่ะ,1024
3528464,38346531,Gotham ตั้งใจจะให้ Jerome Valeska เป็น The Joker ตั้งแต่แรกแล้วหรือเปล่าครับ,"พอดีเพิ่งได้มาดูซีรี่ส์ Gotham ตอนนี้ดูจนถึงซีซั่น3 ยังดูไม่จบ แต่สงสัยว่าทางซีรี่ส์นั้นตั้งใจจะให้ Jerome Valeska เป็น The Joker ในวัยรุ่นตั้งแต่แรกหรือเปล่าครับ (ถึงจะรู้ว่าคนที่จะมาเป็นโจ๊กเกอร์ตัวจริงคือ Jeremiah ซึ่งเป็นพี่น้องฝาแฝดก็เถอะ แต่ผมยังดูไม่ถึง) เพราะเห็นซีซั่น2 Jerome หายไปไม่มีบทเลย อาจเพราะแฟนๆเรียกร้องให้นักแสดงคนนี้กลับมามีบทบาทอีกครั้งหรือเปล่าเพราะแสดงเหมือน Joker , หรือจริงๆได้วางเนื้อเรื่องไว้อยู่แล้วตั้งแต่ซีซั่นแรก และตั้งใจจะให้

In [214]:
all_df[all_df.prompt_tokens<=512][['content_id','title','body','prompt_tokens']]\
    .drop_duplicates().sort_values('prompt_tokens',ascending=False).head(50)

,content_id,title,body,prompt_tokens
4726040,32511452,ใครย้ายค่ายจาก AIS ไป Dtac สำเร็จแล้วบ้างค่ะ เบื่อAIS จะยื้อทำไมค่ะ ?,"ของเราAISเติมเงิน. ย้ายไม่ได้สักที ทั้งทีเช็คข้อมูลกับ AIS ว่าตรงกับที่แจ้งย้ายไว้กับ Dtacโทรเช็คหลายรอบ จนรำคาญแล้ว ส่งเมลล์ไปถาม ก็ตอบแค่ลงทะเบียนแล้ว แย่มากค่ะเช็คทั้งAIS และ Dtac แต่ย้ายไม่ได้เพราะอะไรค่ะ ? เครือข่ายเดิมข้อมูลไม่ตรงกัน ข้อมูลอะไรอีก ?โทรหลายรอบจนเซร็งกับ AIS นี่ย้ายแค่เบอร์เรา เบอร์เดียวนะ เดี๋ยวโมโหจะย้ายเบอร์รายเดือนของที่บ้าน กับที่ทำงานด้วย ควรสู้กันด้วยการตลาดให้เหมาะสมกับเป็น AIS หน่อยค่ะ ยื้อลูกค้าแบบนี้คุณอาจเสียลูกค้าเพิ่มนะค่ะ....ควรทำหรอค่ะ?T_____Tนี่โทรไปแจ้งกับDtac แล้ว นอกจาก ""คุณ "" แล้วดูสิ AIS จะมีอะไรให้ผิดอีกเราทำเรื่องย้ายตั้งแต่24/8/2014 ยังย้ายไม่ได้เลย ตอนแรกคิดว่าเป็นคนเดียว เข้าพันทิพ มีเพื่อนเยอะเลย+5555cc AIS ขอบคุณมากค่ะ(เสียความรู้สึกมากค่ะ )cc dtac โทร1678 2ครั้งแล้วย้ายได้ยังค่ะ ขอบคุณค่ะ",512
1195454,37048003,ต่อมน้ำเหลืองอักเสบบ่อย เป็นอะไรไหมคะ,เราเป็นคนที่ต่อมน้ำเหลืองอักเสบบ่อยมาก โดยเฉพาะบริเวณใต้คาง ลำคอ แถวๆกกหู มันไม่ได้บวมจนน่ากลัวค่ะแต่เวลาเป็นจะรู้สึกปวด บางครั้งตื่นนอนมาเมื่อยไปทั้งหน้าเหมือนคนโดนต่อย พอคลำๆกดดูที่คอคือรู้เลยค่ะว่าบวมอีกแล้ว อักเสบอีกแล้วแรกๆก็กินยา หาหมอ จนไปๆมาๆเราเลิกกินค่ะเพราะเป็นบ่อยมาก ใช้วิธีปล่อยให้หายไปเอง เท่าที่เราสังเกตคือพอเริ่มนอนดึกหน่อยก็จะเป็นค่ะเหมือนสัญญาณเตือนว่าเราพักผ่อนไม่พอ แล้วเราจะเป็นคนที่เป็นแผลที่เท้าบ่อย เท้าแตก เนื่องจากที่บ้านเป็นพื้นปูนแล้วปูนกัดค่ะทุกครั้งที่มีแผลเหมือนจะเป็นส่วนนึงที่ทำให้ต่อมน้ำเหลืองอักเสบเลย แต่เวลาอักเสบก็บวมแค่บริเวณลำคอนะคะ ไม่เคยบวมตามขาหนีบอะไรแบบนี้อยากทราบว่ามีคนไหนเป็นเหมือนเราบ้างคะ เป็นบ่อยจนรำคาญเลย แล้วมีวิธีไหนมั้ยที่จะทำให้ไม่เป็น นอกเหนือจากการพักผ่อนคือเรานอนเฉลี่ย7-8ชม.ทุกวันค่ะ แต่ก็ยังดูเหมือนพักผ่อนไม่พอสักที,512
365179,30812467,Laptop Sony Viao อายุ 3 ปี ไม่อยากซื้อใหม่ทำอะไรกับมันได้บ้างให้ดีและเร็วขึ้น,"เราไม่มีความร้เรื่องคอม ช่วงนี้ต้องใช้ Laptop มากขึ้นหิ้วไปทำงานการใช้งานคือเข้าอินเตอร์เนต พิมพ์งาน Microsoft office ธรรมดา ไม่ใช่งานกราฟฟิกมีเปิด Acrobat และ Photoshop บ้าง1. เพื่อนๆช่วยแนะนำหน่อยว่าเราต้องทำอะไรกับ laptop เดิมที่มีบ้างเพราะรู้สึกว่ามันช้าตอนเปิดเครื่องและตอนจะเข้าดูไฟล์ต่างๆล่าสุดที่เรา Launch Viao Recovery ไป format C drive หมดเลยก็เร็วขึ้นบ้าง2. ต้องเข้าร้านให้ลง Microsoft office 2010 มั้ย ของเดิมเป็น 20073. Anti-Virus หมดอายุแล้ว Mcafee อยากได้ตัวใหม่ให้หาลงเองหรือร้านลงให้ อยากได้ของแท้ค่ะ ตัวไหนดี4. ถ้าต้องไปร้านมีที่แนะนำมั้ย แถว Fortune หลังไมค์มาได้ค่ะSpec LaptopSony VAIO S-Series VPCS116FHCPUIntel Core i5-520M (2.4GHz, 3 MB L3 Cache, 1066 MHz FSB) with Turbo Boost up to 2.93 GHz RAM4G DDR3 (อัพเกรดได้มากถึง 8G)Harddisk320GB (Serial ATA, 5400 rpm)Graphics การ์ดจอแยกNVIDIA GeForce 310M (512MB DDR3) GPU พร้อมเทคโนโลยี CUDA (up to 2303MB of TurboCache)OSWindows 7 Home Premium 64-bit License",512
2386489,31158014,ขอพูดเกี่ยวกับเรื่องหนังไทยหน่อยนะคะ,ขอระบายหน่อยนะคะ คงเป็นเรื่องสุดท้ายที่จะดูหนังไทย หนังไทยมีแต่แนวปัญญาอ่อน รักๆตลกๆ แต่ฝืดมาก แล้วก็แนวผีๆตลก มีอยู่แค่นี้สร้างแนวอื่นไม่เป็น คนไทยก็ชอบไปสนับสนุนหนังไทยไม่มีคุณภาพแบบนี้ ค่ายหนังก็เลยไม่คิดจะพัฒนาให้มันดีกว่านี้ ดูอย่างต้มยำกุ้ง2 บท cg ตัดต่อ เนื้อเรื่อง ห่วยแตกมากๆ นักแสดงก็แสดงได้แข็งทื่อไม่มีความสมจริงสักอย่างเกือบทุกเรื่อง ตั้งแต่องค์บาก จนผ่านมา 10ปีก็ยังย่ำอยู่กับที่ แถมแย่ลงกว่าเดิมอีก ไม่อยากจะเชื่อว่าลงทุน500 ล้าน คิดว่าแค่สร้างกระแสโปรโมตหนังมากกว่า หัดสร้างหนังที่มีความแตกต่างจากเดิมบ้างสิ คนดูเขาเบื่อและเอียนจะแย่อยู่แล้ว สร้างหนังแนวsci-fi อย่างเรื่องgravity ก็ได้ ดูอย่างหนังฝรั่งมีแต่จะพัฒนาขึ้นๆๆ มีทางเดียวที่หนังไทยจะพัฒนาขึ้นกว่านี้ ก็คือคนไทยต้องอย่าไปดูหนังไทยที่ห่วยสิ้นคิด ค่ายหนังมันจะได้คิดและตั้งใจที่จะสร้างหนังให้มันมีคุณภาพ เซงหนังไทยมากๆ,512
670623,31629939,จะจับผู้ทำผิดได้หรือไม่ ขณะนี้ตำรวจแชร์ภาพ “มือปืนถุงป๊อปคอร์น“ ยิงเดือดแยกหลักสี่,"นำเสนอข่าวโดยทีมงาน Sanook.com(8 ก.พ.) ผู้สื่อข่าวรายงานว่า เมื่อช่วงบ่ายที่ผ่านมา โฆษกสำนักงานตำรวจแห่งชาติได้โพสต์ภาพและข้อความลงในเฟซบุ๊ก Policespokensmen เผยแพร่ภาพชายต้องสงสัยเป็นบุคคลที่ใช้อาวุธสงครามยิงบริเวณแยกหลักสี่ เมื่อวันที่ 1 ก.พ. ที่ผ่าน โดยมีเอกลักษณ์ที่ซุกปืนยิงในถุงป๊อปคอร์นทางทีมโ

In [218]:
#min tok 48 (5th percentile) to filter out questions that are too random
#max tok 512 (90th percentile) just to fit 1024 prompt+demo

all_df = all_df[(all_df.prompt_tokens>=48)&(all_df.prompt_tokens<=512)]
all_df.shape, all_df.content_id.nunique()

((3568460, 30), 994334)

## Create Preference Label

In [224]:
more_than_one = all_df.content_id.value_counts().reset_index()
more_than_one = more_than_one[more_than_one.content_id>1]
more_than_one.columns = ['content_id','nb_comments']
more_than_one

,content_id,nb_comments
0,31113669,5
1,38820720,5
2,38819755,5
3,33610623,5
4,33610710,5
...,...,...
935905,30930900,2
935906,34579440,2
935907,36667355,2
935908,38275123,2


In [226]:
all_df = all_df.merge(more_than_one,on='content_id',how='inner')
all_df.shape, all_df.content_id.nunique()

((3510036, 31), 935910)

In [236]:
#sort by nb_interact then comment_tokens; topmost has label 1, otherwise 0
all_df['rnk'] = all_df.sort_values(['nb_interact','comment_tokens'], ascending=[False,False]) \
             .groupby(['content_id']) \
             .cumcount() + 1
all_df['preference_label'] = all_df['rnk'].map(lambda x: 1 if x==1 else 0)

In [261]:
all_df[[
        'content_id','tags','nb_comments','order_comment',
        'title_tokens','body_tokens','comment_tokens',
        'title','body','comment',
        'preference_label','nb_interact','strict']].head(30)

,content_id,tags,nb_comments,order_comment,title_tokens,body_tokens,comment_tokens,title,body,comment,preference_label,nb_interact,strict
0,36188003,"Social Network,Twitter",4,ความคิดเห็นที่ 1,20,66,28,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินการนี้ได้รับการรายงานว่าน่าสงสัย ต้องทำยังไงคะช่วยบอกที???,ส่งหาใครคะ ดีเอ็มต้องโฟโลทั้งสองฝ่ายปะอะ,0,0,0.0
1,36188003,"Social Network,Twitter",4,ความคิดเห็นที่ 2,20,66,52,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินการนี้ได้รับการรายงานว่าน่าสงสัย ต้องทำยังไงคะช่วยบอกที???,ไม่ต้อง follow กันและกัน ก็ส่งได้นะส่งมาแล้วแต่ถ้าอีกฝ่าย block คุณยังไงก็ส่งไม่ได้,0,0,0.0
2,36188003,"Social Network,Twitter",4,ความคิดเห็นที่ 3,20,66,55,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินการนี้ได้รับการรายงานว่าน่าสงสัย ต้องทำยังไงคะช่วยบอกที???,จขกท.แก้ได้ยังค่ะ เราก็เปน ฝากด้วยนะคะ(แต่เราส่งไม่ได้แค่แอคเดียว นอกนั้นปกติค่ะ),1,0,0.0
3,36188003,"Social Network,Twitter",4,ความคิดเห็นที่ 4,20,66,15,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินการนี้ได้รับการรายงานว่าน่าสงสัย ต้องทำยังไงคะช่วยบอกที???,แก้ได้ยังคะ?เราก็เป็น,0,0,0.0
4,36188128,"Freeware,ซอฟต์แวร์,Multimedia Software,ตัดต่อวิดีโอ,Adobe Premiere Pro",2,ความคิดเห็นที่ 2,36,36,37,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับมือใหม่ครับ,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับมือใหม่ครับ,Sony vegas 13.0 //สำหรับผมเเล้วใช้ได้หลากหลาย เเละเข้าใจง่ายดี,0,0,0.0
5,36188128,"Freeware,ซอฟต์แวร์,Multimedia Software,ตัดต่อวิดีโอ,Adobe Premiere Pro",2,ความคิดเห็นที่ 3,36,36,124,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับมือใหม่ครับ,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับมือใหม่ครับ,เอาง่าย ไม่แฮงค์ ไม่หงุดหงิดเรนเดอร์ไวแอฟทันสมัย MAC+FCPXฮอลลี่วูดใช้ครับที่นี่สอนทุกโปรแกรมจึงรู้ความยากง่ายความทันสมัย ความผิดพลาด ของแอฟต่างๆครับสนใจเรียนตัดต่อที่[URL] อยากถามอยากทำ แวะมาเรียนได้ครับ,1,0,0.0
6,36187937,"ชิ้นส่วนคอมพิวเตอร์,จัดสเป็คคอม,คอมพิวเตอร์แบบตั้งโต๊ะ,ฮาร์ดแวร์",5,ความคิดเห็นที่ 1,19,109,45,mainboard ไหนที่รองรับ gtx 950 มั่งครับ,พอดีว่าผมอยากจะเปลี่ยนmainboardอะครับ เพราะเมนบอร์ดผมไม่รองรับ แล้วอยากได้เมนบอร์ดที่รองรับ amd fx-8320eด้วยอะครับ แรม ddr3 ผมจะได้ไม่ต้องเสียค่าแรมใหม่mainboard ผมตอนนี้ gigabyte ga-78lmt-s2,ใครบอกว่าไม่รองรับครับ บอร์ดที่ใส่อยู่ก็มีทุกอย่างที่ต้องการนี่ครับ,0,0,0.0
7,36187937,"ชิ้นส่วนคอมพิวเตอร์,จัดสเป็คคอม,คอมพิวเตอร์แบบตั้งโต๊ะ,ฮาร์ดแวร์",5,ความคิดเห็นที่ 2,19,109,17,mainboard ไหนที่รองรับ gtx 950 มั่งครับ,พอดีว่าผมอยากจะเปลี่ยนmainboardอะครับ เพราะเมนบอร์ดผมไม่รองรับ แล้วอยากได้เมนบอร์ดที่รองรับ amd fx-8320eด้วยอะครับ แรม ddr3 ผมจะได้ไม่ต้องเสียค่าแรมใหม่mainboard ผมตอนนี้ gigabyte ga-78lmt-s2,ทำไมถึงคิดว่ามันไม่รองรับ,0,0,0.0
8,36187937,"ชิ้นส่วนคอมพิวเตอร์,จัดสเป็คคอม,คอมพิวเตอร์แบบตั้งโต๊ะ,ฮาร์ดแวร์",5,ความคิดเห็นที่ 3,19,109,59,mainboard ไหนที่รองรับ gtx 950 มั่งครับ,พอดีว่าผมอยากจะเปลี่ยนmainboardอะครับ เพราะเมนบอร์ดผมไม่รองรับ แล้วอยากได้เมนบอร์ดที่รองรับ amd fx-8320eด้วยอะครับ แรม ddr3 ผมจะได้ไม่ต้องเสียค่าแรมใหม่mainboard ผมตอนนี้ gigabyte ga-78lmt-s2,JIBสาขาไหนครับพนักบ้างคนไม่รู้เรื่องครับ เราเอาตัวอุปกรณ์ ไปให้เทสเลยที่ร้านJIBสามารถเทสได้เลย,0,0,0.0
9,36187937,"ชิ้นส่วนคอมพิวเตอร์,จัดสเป็คคอม,คอมพิวเตอร์แบบตั้งโต๊ะ,ฮาร์ดแวร์",5,ความคิดเห็นที่ 4,19,109,24,mainboard ไหนที่รองรับ gtx 950 มั่งครับ,พอดีว่าผมอยากจะเปลี่ยนmainboardอะครับ เพราะเมนบอร์ดผมไม่รองรับ แล้วอยากได้เมนบอร์ดที่รองรับ amd fx-8320eด้วยอะครับ แรม ddr3 ผมจะได้ไม่ต้องเสียค่าแรมใหม่mainboard ผมตอนนี้ gigabyte ga-78lmt-s2,ถ้า 1050ti เกินงบก็เอา 1050 เฉยๆก็ได้,0,0,0.0


In [251]:
all_df.shape, all_df.content_id.nunique(),\
all_df[all_df.strict==1].shape, all_df[all_df.strict==1].content_id.nunique()

((3510036, 34), 935910, (1586165, 34), 388645)

In [252]:
#see a random thread
all_df[all_df.content_id==31114363][['title','body','comment','nb_interact','like_score_y']]

,title,body,comment,nb_interact,like_score_y
3510019,ลิขสิทธิ์ถ่ายทอดสดฟุตบอลตอนนี้ใครได้อะไรไปบ้าง,ทั้งในประเทศ ต่างประเทศ ทีมชาติ สโมสร ลีก ถ้วย,รวบรวมมาให้gmmFrance - Ligue 1 (2012-2016)Germany - Bundesliga (2012-2015)Japan - J1 League (2012-2013)champienship english (2012-2013)ch7 (ได้สิทธิ์จากเจ้าของสิขสิทธิ์)World Cup Qualifying (2013)ทีมชาติ England (2013-2016)Spain - La Liga (2012-2015)cthEngland - Premier League (2013-2016)CTH acquired live audio visual broadcast rights for all 380 EPL matches per season.Australia - A-League (2013-2014)ch9UEFA Champions League (2012-2015)England - Premier League (2013-2014)UEFA Europa League (2012-2015)trueUEFA Europa League (2012-2015)Italy - Serie A (2012-2015)UEFA Champions League (2012-2015)rsSpain - La Liga (2012-2015)คัดเฉพาะที่น่าสนใจมานะ ขาดหลายลีกอยู่,1,0
3510020,ลิขสิทธิ์ถ่ายทอดสดฟุตบอลตอนนี้ใครได้อะไรไปบ้าง,ทั้งในประเทศ ต่างประเทศ ทีมชาติ สโมสร ลีก ถ้วย,GMM รู้สึกจะได้ลิขสิทธิ์เกมเตะของทีมชาติอิตาลีด้วยมั้งครับ เห็นถ่ายทุกนัด ไม่มีปล่อยให้ชาวบ้านเลยบอลถ้วย Coppa Italia ก็น่าจะยังอยู่กับทรูนะครับ ถ่ายทุกนัดตั้งแต่รอบ 16 ทีม ถึงนัดชิงฯ,0,0


## Package as Dataset

In [ ]:
import json

with open('../data_large/php_json/php_universe.jsonl', 'w', encoding='utf-8') as f:
    for content_id in tqdm(all_df.content_id.unique()):
        d = all_df[all_df.content_id==content_id]
        thread_d = d[[
        'content_id','tags','nb_comments',
        'title_tokens','body_tokens','title','body','strict'
        ]].drop_duplicates().to_dict(orient='records')[0]
        comments = []
        for i,row in d.iterrows():
            comment_d = row[[
                'order_comment',
                'comment_tokens',
                'comment',
                'preference_label',
                'nb_interact']].to_dict()
            comments.append(comment_d)
        thread_d['comments'] = comments
        
        #write to jsonl
        json.dump(thread_d, f, ensure_ascii=False)
        f.write('\n')

  0%|          | 0/935910 [00:00<?, ?it/s]

In [319]:
import json

my_list = []

with open('../data_large/php_json/php_universe.jsonl', 'r', encoding='utf-8') as file:
    for line in tqdm(file):
        item = json.loads(line.strip())
        my_list.append(item)

len(my_list)

0it [00:00, ?it/s]

935910

In [321]:
my_list[-3]

{'content_id': 31113938,
 'tags': 'วอลเลย์บอล',
 'nb_comments': 5,
 'title_tokens': 38,
 'body_tokens': 302,
 'title': 'ณ จุดๆนี้ Xu Yunli ถือว่าเป็น MB เบอร์ 1 ของจีนได้หรือยังครับ ???',
 'body': 'สังเกตุได้ว่า เวลาแข่งกับทีมเก่งๆ อย่างไทย หรือเกาหลี จะเห็นว่ายุนลียืนยาวตลอดเกมส์ป้าหม่า ยังมีเปลี่ยนสลับกับจุงจิงอยู่บ้าง แต่คุณยุนลีนี่ยืนตลอดทั้งเกมส์แม้ว่าจะไม่ตบหายๆ ทำแต้มได้ 100% แบบตอน wgp2012 แต่ส่วนตัวผม ผมคิดว่าก็ยังฝากผีฝากไข้ได้มากกว่า MB ที่เหลือของจีนยังทำแต้มได้เรื่อยๆ และยังถือว่าเป็นของแสลงของทีมไทยคนนึงเลยยังไงขอความเห็นหน่อยนะครับ ว่า คุณนายยุนลีของผม เรียกว่าเป็น MB เบอร์หนึ่งของจีนได้หรือยังคุยกันสนุกๆครับ แลกเปลี่ยนความเห็นกันครับ ด้วยความเคารพ = )#ติ่งยุนลี',
 'strict': 0.0,
 'comments': [{'order_comment': 'ความคิดเห็นที่ 1',
   'comment_tokens': 23,
   'comment': 'เสิร์ฟก็ดี บล๊อคก็ดี ครบนะคนนี้',
   'preference_label': 0,
   'nb_interact': 1},
  {'order_comment': 'ความคิดเห็นที่ 2',
   'comment_tokens': 42,
   'comment': 'น่าจะได้นะ อย่างชิงแชมป์เอเชียล่าสุดก็ได้ร